# **U-Net (3D)**
 ---

<font size = 4> The 3D U-Net was first introduced by [Çiçek et al](https://arxiv.org/abs/1606.06650) for learning dense volumetric segmentations from sparsely annotated ground-truth data building upon the original U-Net architecture by [Ronneberger et al](https://arxiv.org/abs/1505.04597).

<font size = 4>**This particular implementation allows supervised learning between any two types of 3D image data. If you are interested in image segmentation of 2D datasets, you should use the 2D U-Net notebook instead.**

---

<font size = 4>*Disclaimer*:

<font size = 4>This notebook is part of the *Zero-Cost Deep-Learning to Enhance Microscopy* project ([ZeroCostDL4Mic](https://github.com/HenriquesLab/ZeroCostDL4Mic/wiki)) jointly developed by the [Jacquemet](https://cellmig.org/) and [Henriques](https://henriqueslab.github.io/) laboratories and created by Daniel Krentzel. The BioImage Model Zoo export was jointly developed by [Estibaliz Gómez de Mariscal](https://github.com/esgomezm) (deepImageJ team).

<font size = 4>This notebook is laregly based on the following paper:

<font size = 4>[**3D U-Net: Learning Dense Volumetric Segmentation from Sparse Annotation**](https://arxiv.org/pdf/1606.06650.pdf) by Özgün Çiçek *et al.* published on arXiv in 2016

<font size = 4>The following two Python libraries play an important role in the notebook:

1. <font size = 4>[**Elasticdeform**](https://github.com/gvtulder/elasticdeform)
 by Gijs van Tulder was used to augment the 3D training data using elastic grid-based deformations as described in the original 3D U-Net paper.

2. <font size = 4>[**Tifffile**](https://github.com/cgohlke/tifffile) by Christoph Gohlke is a great library for reading and writing TIFF files.

3. <font size = 4>[**Imgaug**](https://github.com/aleju/imgaug) by Alexander Jung *et al.* is an amazing library for image augmentation in machine learning - it is the most complete and extensive image augmentation package I have found to date.

<font size = 4>The [example dataset](https://www.epfl.ch/labs/cvlab/data/data-em/) represents a 5x5x5µm section taken from the CA1 hippocampus region of the brain with annotated mitochondria and was acquired by Graham Knott and Marco Cantoni at EPFL.

<font size = 4>The guidelines to use the trained network in ImageJ with deepImageJ are given in the following paper:

<font size = 4>**DeepImageJ: a user-friendly environment to run deep learning models in ImageJ**, bioRxiv (2019) by *Estibaliz Gómez-de-Mariscal, Carlos García-López-de-Haro, Wei Ouyang, Laurène Donati, Emma Lundberg, Michael Unser, Arrate Muñoz-Barrutia and Daniel Sage* (https://doi.org/10.1101/799270)

<font size = 4>**Please also cite the original paper and relevant Python libraries when using or developing this notebook.**

# **How to use this notebook?**

---

<font size = 4>Video describing how to use ZeroCostDL4Mic notebooks are available on youtube:
  - [**Video 1**](https://www.youtube.com/watch?v=GzD2gamVNHI&feature=youtu.be): Full run through of the workflow to obtain the notebooks and the provided test datasets as well as a common use of the notebook
  - [**Video 2**](https://www.youtube.com/watch?v=PUuQfP5SsqM&feature=youtu.be): Detailed description of the different sections of the notebook


---
### **Structure of a notebook**

<font size = 4>The notebook contains two types of cells:  

<font size = 4>**Text cells** provide information and can be modified by double-clicking the cell. You are currently reading a text cell. You can create a new one by clicking `+ Text`.

<font size = 4>**Code cells** contain code which can be modfied by selecting the cell. To execute the cell, move your cursor to the `[]`-symbol on the left side of the cell (a play button should appear). Click it to execute the cell. Once the cell is fully executed, the animation stops. You can create a new coding cell by clicking `+ Code`.

---
### **Table of contents, Code snippets** and **Files**

<font size = 4>Three tabs are located on the upper left side of the notebook:

1. <font size = 4>*Table of contents* contains the structure of the notebook. Click the headers to move quickly between sections.

2. <font size = 4>*Code snippets* provides a wide array of example code specific to Google Colab. You can ignore this when using this notebook.

3. <font size = 4>*Files* displays the current working directory. We will mount your Google Drive in Section 1.2. so that you can access your files and save them permanently.

<font size = 4>**Important:** All uploaded files are purged once the runtime ends.

<font size = 4>**Note:** The directory *sample data* in *Files* contains default files. Do not upload anything there!

---
### **Making changes to the notebook**

<font size = 4>**You can make a copy** of the notebook and save it to your Google Drive by clicking *File* -> *Save a copy in Drive*.

<font size = 4>To **edit a cell**, double click on the text. This will either display the source code (in code cells) or the [markdown](https://colab.research.google.com/notebooks/markdown_guide.ipynb#scrollTo=70pYkR9LiOV0) (in text cells).
You can use `#` in code cells to comment out parts of the code. This allows you to keep the original piece of code while not executing it.

# **0. Before getting started**
---

<font size = 4>As the network operates in three dimensions, certain consideration should be given to correctly pre-processing the data. Ensure that the structure of interest does not substantially change between slices - image volumes with isotropic pixelsizes are ideal for this architecture.

<font size = 4>Each image volume must be provided as an **8-bit** or **binary multipage TIFF file** to maintain the correct ordering of individual image slices. If more than one image volume has been annotated, source and target files must be named identically and placed in separate directories. In case only one image volume has been annotated, source and target file do not have to be placed in separate directories and can be named differently, as long as their paths are explicitly provided in Section 3.

<font size = 4>**Prepare two datasets** (*training* and *testing*) for quality control puproses. Make sure that the *testing* dataset does not overlap with the *training* dataset and is ideally sourced from a different acquisiton and sample to ensure robustness of the trained model.


---


### **Directory structure**

<font size = 4>Make sure to adhere to one of the following directory structures. If only one annotated training volume exists, choose the first structure. In case more than one training volume is available, choose the second structure.

<font size = 4>**Structure 1:** Only one training volume

```
path/to/directory/with/one/training/volume
│--training_source.tif
│--training_target.tif
|   
│--testing_source.tif
|--testing_target.tif
|
|--data_to_predict_on.tif
|--prediction_results.tif
```

<font size = 4>**Structure 2:** Various training volumes

```
path/to/directory/with/various/training/volumes
│--testing_source.tif
|--testing_target.tif
|
└───training
|   └───source
|   |   |--training_volume_one.tif
|   |   |--training_volume_two.tif
|   |   |--...
|   |   |--training_volume_n.tif
|   |
|   └───target
|       |--training_volume_one.tif
|       |--training_volume_two.tif
|       |--...
|       |--training_volume_n.tif
|
|--data_to_predict_on.tif
|--prediction_results.tif
```

<font size = 4>**Note:** Naming directories is completely up to you, as long as the paths are correctly specified throughout the notebook.


---


### **Important note**

* <font size = 4>If you wish to **Train a network from scratch** using your own dataset (and we encourage everyone to do so), you will need to run **Sections 1 - 4**, then use **Section 5** to assess the quality of your model and **Section 6** to run predictions using the model that you trained.

* <font size = 4>If you wish to **Evaluate your model** using a model previously generated and saved on your Google Drive, you will only need to run **Sections 1 and 2** to set up the notebook, then use **Section 5** to assess the quality of your model.

* <font size = 4> If you only wish to **Run predictions** using a model previously generated and saved on your Google Drive, you will only need to run **Sections 1 and 2** to set up the notebook, then use **Section 6** to run the predictions on the desired model.
---

# **1. Install 3D U-Net dependencies**
---


## **1.1. Install key dependencies**
---
<font size = 4>

In [ ]:
#@markdown ##Play to install U-Net dependencies
# Install packages which are not included in Google Colab


print("We are installing imgaug, elasticdeform, tifffile, fdf2, bioimageio.core, bioimageio.spec, pydantic and TensorFlow v2.15.0.")
print("This might takes some minutes ...")

# Package for data augmentation
!pip install -q imgaug==0.4.0

# Package for image deformation
!pip install -q elasticdeform==0.5.0

# Package to read .TIF files
!pip install -q tifffile==2024.9.20

# Package for generating PDF documents
!pip install -q fpdf2==2.8.1

# The [bioimageio] extras are installed for bioimageio model support
!pip install -q bioimageio.core==0.7.0
!pip install -q bioimageio.spec==0.5.3.5

# Install fixed version of pydantic to avoid bugs on 2.10.0
!pip install -q pydantic==2.9.2

# TensorFlow 2.15 and it's corresponding dependencies to avoid errors
!pip install -q orbax-checkpoint==0.3.0
!pip install -q tensorstore==0.1.40
!pip install -q tf-keras==2.15
!pip install -q tensorflow==2.15

print("Done!")

## **1.2. Load key dependencies**
---
<font size = 4>

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

#@markdown ##Load key 3D U-Net dependencies and instantiate network
Notebook_version = '2.3.1'
Network = 'U-Net (3D)'

from builtins import any as b_any

class CheckpointError(Exception):
    pass

def get_requirements_path():
    # Store requirements file in 'contents' directory
    current_dir = os.getcwd()
    dir_count = current_dir.count('/') - 1
    path = '../' * (dir_count) + 'requirements.txt'
    return path

def filter_files(file_list, filter_list):
    filtered_list = []
    for fname in file_list:
        if b_any(fname.split('==')[0] in s for s in filter_list):
            filtered_list.append(fname)
    return filtered_list

def build_requirements_file(before, after):
    path = get_requirements_path()

    # Exporting requirements.txt for local run
    !pip freeze > $path

    # Get minimum requirements file
    df = pd.read_csv(path)
    mod_list = [m.split('.')[0] for m in after if not m in before]
    req_list_temp = df.values.tolist()
    req_list = [x[0] for x in req_list_temp]

    # Replace with package name and handle cases where import name is different to module name
    mod_name_list = [['sklearn', 'scikit-learn'], ['skimage', 'scikit-image']]
    mod_replace_list = [[x[1] for x in mod_name_list] if s in [x[0] for x in mod_name_list] else s for s in mod_list]
    filtered_list = filter_files(req_list, mod_replace_list)

    file=open(path,'w')
    for item in filtered_list:
        file.writelines(item)

    file.close()

# Function to check source and target file paths include base path
def check_base_path(base_path, path_to_data):
  if base_path not in path_to_data:
    if path_to_data[0] == '/':
      path_to_data = path_to_data[1:]
    path_to_data = os.path.join(base_path, path_to_data)
  return path_to_data

import sys
before = [str(m) for m in sys.modules]

#Put the imported code and libraries here

from pip._internal.operations.freeze import freeze
from fpdf import FPDF, HTMLMixin
import matplotlib.pyplot as plt
from ipywidgets import interact
from tqdm.notebook import tqdm
from skimage import transform
from datetime import datetime
from shutil import rmtree
from pathlib import Path
from skimage import io
from glob import glob

import imgaug.augmenters as iaa
import ipywidgets as widgets
import elasticdeform
import pandas as pd
import numpy as np
import subprocess
import matplotlib
import requests
import tifffile
import shutil
import random
import math
import time
import csv
import os

# Suppressing some warnings
import warnings
warnings.filterwarnings('ignore')

# Import all TensorFlow dependencies to create the model
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.utils import Sequence
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, Callback
from tensorflow.keras.layers import Conv3D, BatchNormalization, ReLU, MaxPooling3D, Conv3DTranspose, Input, Concatenate

print("TensorFlow version: {}".format(tf.__version__))

# BioImage Model Zoo dependencies
import bioimageio.core as bioimageio_core
from bioimageio.core.digest_spec import create_sample_for_model

import bioimageio.spec
import bioimageio.spec.model.v0_5 as bioimageio_spec
from bioimageio.spec import save_bioimageio_package
from bioimageio.spec._internal.io import FileDescr
from bioimageio.spec.utils import download
from bioimageio.spec.pretty_validation_errors import enable_pretty_validation_errors_in_ipynb

enable_pretty_validation_errors_in_ipynb()

print("Dependencies installed and imported.")

#Create a variable to get and store relative base path
base_path = os.getcwd()

# BMZ model export functions
def make_author(author_input_info: str):
    """
    Create an Author object from a string input.

    Args:
        author_input_info: A string containing the author's name and affiliation.

    Returns:
        An Author object
    """
    if author_input_info.strip() == '':
        return None

    auth_order = ['name', 'affiliation', 'email', 'orcid', 'github_user']
    auth_dict = {}

    auth_info_split = author_input_info.split(',')

    for i in range(len(auth_info_split)):
        if auth_info_split[i].strip() == 'None' or auth_info_split[i].strip() == '':
            continue
        else:
            auth_dict[auth_order[i]] = auth_info_split[i].strip()

    return bioimageio_spec.Author(**auth_dict)

def make_maintainer(maintainer_input_info: str):
    """
    Create an Author object from a string input.

    Args:
        author_input_info: A string containing the author's name and affiliation.

    Returns:
        An Author object
    """
    maint_order = [ 'github_user', 'name', 'affiliation', 'email', 'orcid']
    maint_dict = {}

    maint_info_split = maintainer_input_info.split(',')

    for i in range(len(maint_info_split)):
        if maint_info_split[i].strip() == 'None' or maint_info_split[i].strip() == '':
            continue
        else:
            maint_dict[maint_order[i]] = maint_info_split[i].strip()

    return bioimageio_spec.Maintainer(**maint_dict)

# Define MultiPageTiffGenerator class
class MultiPageTiffGenerator(Sequence):

    def __init__(self,
                 source_path,
                 target_path,
                 batch_size=1,
                 shape=(128,128,32,1),
                 augment=False,
                 augmentations=[],
                 deform_augment=False,
                 deform_augmentation_params=(5,3,4),
                 val_split=0.2,
                 is_val=False,
                 random_crop=True,
                 downscale=1,
                 binary_target=False):

        # If directory with various multi-page tiffiles is provided read as list
        if os.path.isfile(source_path):
            self.dir_flag = False
            self.source = tifffile.imread(source_path)
            if binary_target:
                self.target = tifffile.imread(target_path).astype(bool)
            else:
                self.target = tifffile.imread(target_path)

        elif os.path.isdir(source_path):
            self.dir_flag = True
            self.source_dir_list = glob(os.path.join(source_path, '*'))
            self.target_dir_list = glob(os.path.join(target_path, '*'))

            self.source_dir_list.sort()
            self.target_dir_list.sort()

        self.shape = shape
        self.batch_size = batch_size
        self.augment = augment
        self.val_split = val_split
        self.is_val = is_val
        self.random_crop = random_crop
        self.downscale = downscale
        self.binary_target = binary_target
        self.deform_augment = deform_augment
        self.on_epoch_end()

        if self.augment:
            # pass list of augmentation functions
            self.seq = iaa.Sequential(augmentations, random_order=True) # apply augmenters in random order
        if self.deform_augment:
            self.deform_sigma, self.deform_points, self.deform_order = deform_augmentation_params

    def __len__(self):
        # If various multi-page tiff files provided sum all images within each
        if self.augment:
            augment_factor = 4
        else:
            augment_factor = 1

        if self.dir_flag:
            num_of_imgs = 0
            for tiff_path in self.source_dir_list:
                num_of_imgs += tifffile.imread(tiff_path).shape[0]
            xy_shape = tifffile.imread(self.source_dir_list[0]).shape[1:]

            if self.is_val:
                if self.random_crop:
                    crop_volume = self.shape[0] * self.shape[1] * self.shape[2]
                    volume = xy_shape[0] * xy_shape[1] * self.val_split * num_of_imgs
                    return math.floor(augment_factor * volume / (crop_volume * self.batch_size * self.downscale))
                else:
                    return math.floor(self.val_split * num_of_imgs / self.batch_size)
            else:
                if self.random_crop:
                    crop_volume = self.shape[0] * self.shape[1] * self.shape[2]
                    volume = xy_shape[0] * xy_shape[1] * (1 - self.val_split) * num_of_imgs
                    return math.floor(augment_factor * volume / (crop_volume * self.batch_size * self.downscale))

                else:
                    return math.floor(augment_factor*(1 - self.val_split) * num_of_imgs/self.batch_size)
        else:
            if self.is_val:
                if self.random_crop:
                    crop_volume = self.shape[0] * self.shape[1] * self.shape[2]
                    volume = self.source.shape[0] * self.source.shape[1] * self.val_split * self.source.shape[2]
                    return math.floor(augment_factor * volume / (crop_volume * self.batch_size * self.downscale))
                else:
                    return math.floor((self.val_split * self.source.shape[0] / self.batch_size))
            else:
                if self.random_crop:
                    crop_volume = self.shape[0] * self.shape[1] * self.shape[2]
                    volume = self.source.shape[0] * self.source.shape[1] * (1 - self.val_split) * self.source.shape[2]
                    return math.floor(augment_factor * volume / (crop_volume * self.batch_size * self.downscale))
                else:
                    return math.floor(augment_factor * (1 - self.val_split) * self.source.shape[0] / self.batch_size)

    def __getitem__(self, idx):
        source_batch = np.empty((self.batch_size,
                                 self.shape[0],
                                 self.shape[1],
                                 self.shape[2],
                                 self.shape[3]))
        target_batch = np.empty((self.batch_size,
                                 self.shape[0],
                                 self.shape[1],
                                 self.shape[2],
                                 self.shape[3]))

        for batch in range(self.batch_size):
            # Modulo operator ensures IndexError is avoided
            stack_start = self.batch_list[(idx+batch*self.shape[2])%len(self.batch_list)]

            if self.dir_flag:
                self.source = tifffile.imread(self.source_dir_list[stack_start[0]])
                if self.binary_target:
                    self.target = tifffile.imread(self.target_dir_list[stack_start[0]]).astype(bool)
                else:
                    self.target = tifffile.imread(self.target_dir_list[stack_start[0]])

            src_list = []
            tgt_list = []
            for i in range(stack_start[1], stack_start[1]+self.shape[2]):
                src = self.source[i]
                src = transform.downscale_local_mean(src, (self.downscale, self.downscale))
                if not self.random_crop:
                    src = transform.resize(src, (self.shape[0], self.shape[1]), mode='constant', preserve_range=True)
                src = self._min_max_scaling(src)
                src_list.append(src)

                tgt = self.target[i]
                tgt = transform.downscale_local_mean(tgt, (self.downscale, self.downscale))
                if not self.random_crop:
                    tgt = transform.resize(tgt, (self.shape[0], self.shape[1]), mode='constant', preserve_range=True)
                if not self.binary_target:
                    tgt = self._min_max_scaling(tgt)
                tgt_list.append(tgt)

            if self.random_crop:
                if src.shape[0] == self.shape[0]:
                    x_rand = 0
                if src.shape[1] == self.shape[1]:
                    y_rand = 0
                if src.shape[0] > self.shape[0]:
                    x_rand = np.random.randint(src.shape[0] - self.shape[0])
                if src.shape[1] > self.shape[1]:
                    y_rand = np.random.randint(src.shape[1] - self.shape[1])
                if src.shape[0] < self.shape[0] or src.shape[1] < self.shape[1]:
                    raise ValueError('Patch shape larger than (downscaled) source shape')

            for i in range(self.shape[2]):
                if self.random_crop:
                    src = src_list[i]
                    tgt = tgt_list[i]
                    src_crop = src[x_rand:self.shape[0]+x_rand, y_rand:self.shape[1]+y_rand]
                    tgt_crop = tgt[x_rand:self.shape[0]+x_rand, y_rand:self.shape[1]+y_rand]
                else:
                    src_crop = src_list[i]
                    tgt_crop = tgt_list[i]

                source_batch[batch,:,:,i,0] = src_crop
                target_batch[batch,:,:,i,0] = tgt_crop

        if self.augment:
            # On-the-fly data augmentation
            source_batch, target_batch = self.augment_volume(source_batch, target_batch)

            # Data augmentation by reversing stack
            if np.random.random() > 0.5:
                source_batch, target_batch = source_batch[::-1], target_batch[::-1]

            # Data augmentation by elastic deformation
            if np.random.random() > 0.5 and self.deform_augment:
                source_batch, target_batch = self.deform_volume(source_batch, target_batch)

            if not self.binary_target:
                target_batch = self._min_max_scaling(target_batch)

            return self._min_max_scaling(source_batch), target_batch

        else:
            return source_batch, target_batch

    def on_epoch_end(self):
        # Validation split performed here
        self.batch_list = []
        # Create batch_list of all combinations of tifffile and stack position
        if self.dir_flag:
            for i in range(len(self.source_dir_list)):
                num_of_pages = tifffile.imread(self.source_dir_list[i]).shape[0]
                if self.is_val:
                    start_page = num_of_pages-math.floor(self.val_split*num_of_pages)
                    for j in range(start_page, num_of_pages-self.shape[2]):
                      self.batch_list.append([i, j])
                else:
                    last_page = math.floor((1-self.val_split)*num_of_pages)
                    for j in range(last_page-self.shape[2]):
                        self.batch_list.append([i, j])
        else:
            num_of_pages = self.source.shape[0]
            if self.is_val:
                start_page = num_of_pages-math.floor(self.val_split*num_of_pages)
                for j in range(start_page, num_of_pages-self.shape[2]):
                    self.batch_list.append([0, j])

            else:
                last_page = math.floor((1-self.val_split)*num_of_pages)
                for j in range(last_page-self.shape[2]):
                    self.batch_list.append([0, j])

        if self.is_val and (len(self.batch_list) <= 0):
            raise ValueError('validation_split too small! Increase val_split or decrease z-depth')
        random.shuffle(self.batch_list)

    def _min_max_scaling(self, data):
        n = data - np.min(data)
        d = np.max(data) - np.min(data)

        return n/d

    def class_weights(self):
        ones = 0
        pixels = 0

        if self.dir_flag:
            for i in range(len(self.target_dir_list)):
                tgt = tifffile.imread(self.target_dir_list[i]).astype(bool)
                ones += np.sum(tgt)
                pixels += tgt.shape[0]*tgt.shape[1]*tgt.shape[2]
        else:
          ones = np.sum(self.target)
          pixels = self.target.shape[0]*self.target.shape[1]*self.target.shape[2]
        p_ones = ones/pixels
        p_zeros = 1-p_ones

        # Return swapped probability to increase weight of unlikely class
        return p_ones, p_zeros

    def deform_volume(self, src_vol, tgt_vol):
        [src_dfrm, tgt_dfrm] = elasticdeform.deform_random_grid([src_vol, tgt_vol],
                                                                axis=(1, 2, 3),
                                                                sigma=self.deform_sigma,
                                                                points=self.deform_points,
                                                                order=self.deform_order)
        if self.binary_target:
            tgt_dfrm = tgt_dfrm > 0.1

        return self._min_max_scaling(src_dfrm), tgt_dfrm

    def augment_volume(self, src_vol, tgt_vol):

        src_vol_aug = np.empty(src_vol.shape)
        tgt_vol_aug = np.empty(tgt_vol.shape)

        for i in range(src_vol.shape[3]):
            src_aug_z, tgt_aug_z = self.seq(images=src_vol[:,:,:,i,0].astype('float16'),
                                                                      segmentation_maps=np.expand_dims(tgt_vol[:,:,:,i,0].astype(bool), axis=-1))
            src_vol_aug[:,:,:,i,0] = src_aug_z
            tgt_vol_aug[:,:,:,i,0] = np.squeeze(tgt_aug_z)
        return self._min_max_scaling(src_vol_aug), tgt_vol_aug

    def sample_augmentation(self, idx):
        src, tgt = self.__getitem__(idx)

        src_aug, tgt_aug = self.augment_volume(src, tgt)

        if self.deform_augment:
            src_aug, tgt_aug = self.deform_volume(src_aug, tgt_aug)

        return src_aug, tgt_aug

# Define custom loss and dice coefficient
def dice_coefficient(y_true, y_pred):
    eps = 1e-6
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)

    # Cast both to float 32 to avoid issues
    y_true_f = K.cast(y_true_f, tf.float32)
    y_true_f = K.cast(y_true_f, tf.float32)
    
    intersection = K.sum(y_true_f*y_pred_f)

    return (2.*intersection)/(K.sum(y_true_f*y_true_f)+K.sum(y_pred_f*y_pred_f)+eps)

def weighted_binary_crossentropy(zero_weight, one_weight):
    def _weighted_binary_crossentropy(y_true, y_pred):
        binary_crossentropy = K.binary_crossentropy(y_true, y_pred)

        weight_vector = y_true*one_weight+(1.-y_true)*zero_weight
        weighted_binary_crossentropy = weight_vector*binary_crossentropy

        return K.mean(weighted_binary_crossentropy)

    return _weighted_binary_crossentropy

# Custom callback showing sample prediction
class SampleImageCallback(Callback):

    def __init__(self, model, sample_data, model_path, save=False):
        self.model = model
        self.sample_data = sample_data
        self.model_path = model_path
        self.save = save

    def on_epoch_end(self, epoch, logs={}):
        sample_predict = self.model.predict_on_batch(self.sample_data)

        f=plt.figure(figsize=(16,8))
        plt.subplot(1,2,1)
        plt.imshow(self.sample_data[0,:,:,0,0], interpolation='nearest', cmap='gray')
        plt.title('Sample source')
        plt.axis('off');

        plt.subplot(1,2,2)
        plt.imshow(sample_predict[0,:,:,0,0], interpolation='nearest', cmap='magma')
        plt.title('Predicted target')
        plt.axis('off');

        plt.show()

        if self.save:
            plt.savefig(self.model_path + '/epoch_' + str(epoch+1) + '.png')

# Define Unet3D class
class Unet3D:

    def __init__(self,
                 shape=(256,256,16,1)):
        if isinstance(shape, str):
            shape = eval(shape)

        self.shape = shape

        input_tensor = Input(self.shape, name='input')

        self.model = self.unet_3D(input_tensor)

    def down_block_3D(self, input_tensor, filters):
        x = Conv3D(filters=filters, kernel_size=(3,3,3), padding='same')(input_tensor)
        x = BatchNormalization()(x)
        x = ReLU()(x)

        x = Conv3D(filters=filters*2, kernel_size=(3,3,3), padding='same')(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)

        return x

    def up_block_3D(self, input_tensor, concat_layer, filters):
        x = Conv3DTranspose(filters, kernel_size=(2,2,2), strides=(2,2,2))(input_tensor)

        x = Concatenate()([x, concat_layer])

        x = Conv3D(filters=filters, kernel_size=(3,3,3), padding='same')(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)

        x = Conv3D(filters=filters*2, kernel_size=(3,3,3), padding='same')(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)

        return x

    def unet_3D(self, input_tensor, filters=32):
        d1 = self.down_block_3D(input_tensor, filters=filters)
        p1 = MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2), data_format='channels_last')(d1)
        d2 = self.down_block_3D(p1, filters=filters*2)
        p2 = MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2), data_format='channels_last')(d2)
        d3 = self.down_block_3D(p2, filters=filters*4)
        p3 = MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2), data_format='channels_last')(d3)

        d4 = self.down_block_3D(p3, filters=filters*8)

        u1 = self.up_block_3D(d4, d3, filters=filters*4)
        u2 = self.up_block_3D(u1, d2, filters=filters*2)
        u3 = self.up_block_3D(u2, d1, filters=filters)

        output_tensor = Conv3D(filters=1, kernel_size=(1,1,1), activation='sigmoid')(u3)

        return Model(inputs=[input_tensor], outputs=[output_tensor])

    def summary(self):
        return self.model.summary()

    # Pass generators instead
    def train(self,
              epochs,
              batch_size,
              train_generator,
              val_generator,
              model_path,
              model_name,
              optimizer='adam',
              learning_rate=0.001,
              loss='weighted_binary_crossentropy',
              metrics='dice',
              ckpt_period=1,
              save_best_ckpt_only=False,
              ckpt_path=None):

        class_weight_zero, class_weight_one = train_generator.class_weights()

        if loss == 'weighted_binary_crossentropy':
            loss = weighted_binary_crossentropy(class_weight_zero, class_weight_one)

        if metrics == 'dice':
            metrics = dice_coefficient

        if optimizer == 'adam':
            optimizer = Adam(learning_rate=learning_rate)
        elif optimizer == 'sgd':
            optimizer = SGD(learning_rate=learning_rate)
        elif optimizer == 'rmsprop':
            optimizer = RMSprop(learning_rate=learning_rate)

        self.model.compile(optimizer=optimizer,
                           loss=loss,
                           metrics=[metrics])

        if ckpt_path is not None:
            self.model.load_weights(ckpt_path)

        full_model_path = os.path.join(model_path, model_name)

        if not os.path.exists(full_model_path):
            os.makedirs(full_model_path)

        log_dir = full_model_path + '/Quality Control'

        if not os.path.exists(log_dir):
            os.makedirs(log_dir)

        ckpt_dir =  full_model_path + '/ckpt'

        if not os.path.exists(ckpt_dir):
            os.makedirs(ckpt_dir)

        csv_out_name = log_dir + '/training_evaluation.csv'
        if ckpt_path is None:
            csv_logger = CSVLogger(csv_out_name)
        else:
            csv_logger = CSVLogger(csv_out_name, append=True)

        if save_best_ckpt_only:
            ckpt_name = ckpt_dir + '/' + model_name + '.hdf5'
        else:
            ckpt_name = ckpt_dir + '/' + model_name + '_epoch_{epoch:02d}_val_loss_{val_loss:.4f}.hdf5'

        model_ckpt = ModelCheckpoint(ckpt_name,
                                     verbose=1,
                                     save_freq=ckpt_period,
                                     save_best_only=save_best_ckpt_only,
                                     save_weights_only=True)

        sample_batch, __ = val_generator.__getitem__(random.randint(0, len(val_generator)))

        sample_img = SampleImageCallback(self.model,
                                         sample_batch,
                                         model_path)

        self.model.fit(train_generator,
                       validation_data=val_generator,
                       validation_steps=math.floor(len(val_generator)/batch_size),
                       epochs=epochs,
                       callbacks=[csv_logger,
                                  model_ckpt,
                                  sample_img])

        last_ckpt_name = ckpt_dir + '/' + model_name + '_last.hdf5'
        self.model.save_weights(last_ckpt_name)

    def _min_max_scaling(self, data):
        n = data - np.min(data)
        d = np.max(data) - np.min(data)

        return n/d

    def predict(self,
                input,
                ckpt_path,
                z_range=None,
                downscaling=None,
                true_patch_size=None):

        self.model.load_weights(ckpt_path)

        if isinstance(downscaling, str):
            downscaling = eval(downscaling)

        if math.isnan(downscaling):
            downscaling = None

        if isinstance(true_patch_size, str):
            true_patch_size = eval(true_patch_size)

        if not isinstance(true_patch_size, tuple):
            if math.isnan(true_patch_size):
                true_patch_size = None

        if isinstance(input, str):
            src_volume = tifffile.imread(input)
        elif isinstance(input, np.ndarray):
            src_volume = input
        else:
            raise TypeError('Input is not path or numpy array!')

        in_size = src_volume.shape

        if downscaling or true_patch_size is not None:
            x_scaling = 0
            y_scaling = 0

            if true_patch_size is not None:
                x_scaling += true_patch_size[0]/self.shape[0]
                y_scaling += true_patch_size[1]/self.shape[1]
            if downscaling is not None:
                x_scaling += downscaling
                y_scaling += downscaling

            src_list = []
            for i in range(src_volume.shape[0]):
                 src_list.append(transform.downscale_local_mean(src_volume[i], (int(x_scaling), int(y_scaling))))
            src_volume = np.array(src_list)

        if z_range is not None:
            src_volume = src_volume[z_range[0]:z_range[1]]

        src_volume = self._min_max_scaling(src_volume)

        src_array = np.zeros((1,
                              math.ceil(src_volume.shape[1]/self.shape[0])*self.shape[0],
                              math.ceil(src_volume.shape[2]/self.shape[1])*self.shape[1],
                              math.ceil(src_volume.shape[0]/self.shape[2])*self.shape[2],
                              self.shape[3]))

        for i in range(src_volume.shape[0]):
            src_array[0,:src_volume.shape[1],:src_volume.shape[2],i,0] = src_volume[i]

        pred_array = np.empty(src_array.shape)
        print(src_volume.dtype)
        for i in range(math.ceil(src_volume.shape[1]/self.shape[0])):
          for j in range(math.ceil(src_volume.shape[2]/self.shape[1])):
            for k in range(math.ceil(src_volume.shape[0]/self.shape[2])):
                pred_temp = self.model.predict(src_array[:,
                                                         i*self.shape[0]:i*self.shape[0]+self.shape[0],
                                                         j*self.shape[1]:j*self.shape[1]+self.shape[1],
                                                         k*self.shape[2]:k*self.shape[2]+self.shape[2]])
                pred_array[:,
                           i*self.shape[0]:i*self.shape[0]+self.shape[0],
                           j*self.shape[1]:j*self.shape[1]+self.shape[1],
                           k*self.shape[2]:k*self.shape[2]+self.shape[2]] = pred_temp

        pred_volume = np.rollaxis(np.squeeze(pred_array), -1)[:src_volume.shape[0],:src_volume.shape[1],:src_volume.shape[2]]

        if downscaling is not None:
            pred_list = []
            for i in range(pred_volume.shape[0]):
                 pred_list.append(transform.resize(pred_volume[i], (in_size[1], in_size[2]), preserve_range=True))
            pred_volume = np.array(pred_list)

        return pred_volume

def pdf_export(trained = False, augmentation = False, pretrained_model = False):
  class MyFPDF(FPDF, HTMLMixin):
    pass

  pdf = MyFPDF()
  pdf.add_page()
  pdf.set_right_margin(-1)
  pdf.set_font("Arial", size = 11, style='B')

  day = datetime.now()
  datetime_str = str(day)[0:10]

  Header = 'Training report for '+Network+' model ('+model_name+')\nDate: '+datetime_str
  pdf.multi_cell(180, 5, txt = Header, align = 'L')
  pdf.ln(1)

  # add another cell
  if trained:
    training_time = "Training time: "+str(hour)+ "hour(s) "+str(mins)+"min(s) "+str(round(sec))+"sec(s)"
    pdf.cell(190, 5, txt = training_time, ln = 1, align='L')
  pdf.ln(1)

  Header_2 = 'Information for your materials and methods:'
  pdf.cell(190, 5, txt=Header_2, ln=1, align='L')
  pdf.ln(1)

  all_packages = ''
  for requirement in freeze(local_only=True):
    all_packages = all_packages+requirement+', '
  #print(all_packages)

  #Main Packages
  main_packages = ''
  version_numbers = []
  for name in ['tensorflow','numpy','keras']:
    find_name=all_packages.find(name)
    main_packages = main_packages+all_packages[find_name:all_packages.find(',',find_name)]+', '
    #Version numbers only here:
    version_numbers.append(all_packages[find_name+len(name)+2:all_packages.find(',',find_name)])

  try:
    cuda_version = subprocess.run(["nvcc","--version"],stdout=subprocess.PIPE)
    cuda_version = cuda_version.stdout.decode('utf-8')
    cuda_version = cuda_version[cuda_version.find(', V')+3:-1]
  except:
    cuda_version = ' - No cuda found - '
  try:
    gpu_name = subprocess.run(["nvidia-smi"],stdout=subprocess.PIPE)
    gpu_name = gpu_name.stdout.decode('utf-8')
    gpu_name = gpu_name[gpu_name.find('Tesla'):gpu_name.find('Tesla')+10]
  except:
    gpu_name = ' - No GPU found - '
  #print(cuda_version[cuda_version.find(', V')+3:-1])
  #print(gpu_name)

  if os.path.isdir(training_source):
    shape = io.imread(training_source+'/'+os.listdir(training_source)[0]).shape
  elif os.path.isfile(training_source):
    shape = io.imread(training_source).shape
  else:
    print('Cannot read training data.')

  dataset_size = len(train_generator)

  text = 'The '+Network+' model was trained from scratch for '+str(number_of_epochs)+' epochs on '+str(dataset_size)+' paired image patches (image dimensions: '+str(shape)+', patch size: ('+str(patch_size)+') with a batch size of '+str(batch_size)+' and a '+loss_function+' loss function, using the '+Network+' ZeroCostDL4Mic notebook (v '+Notebook_version[0]+') (von Chamier & Laine et al., 2020). Key python packages used include tensorflow (v '+version_numbers[0]+'), keras (v '+version_numbers[2]+'), numpy (v '+version_numbers[1]+'), cuda (v '+cuda_version+'). The training was accelerated using a '+gpu_name+'GPU.'

  if pretrained_model:
    text = 'The '+Network+' model was trained for '+str(number_of_epochs)+' epochs on '+str(dataset_size)+' paired image patches (image dimensions: '+str(shape)+', patch_size: '+str(patch_size)+') with a batch size of '+str(batch_size)+' and a '+loss_function+' loss function, using the '+Network+' ZeroCostDL4Mic notebook (v '+Notebook_version[0]+') (von Chamier & Laine et al., 2020). The model was retrained from a pretrained model. Key python packages used include tensorflow (v '+version_numbers[0]+'), keras (v '+version_numbers[2]+'), numpy (v '+version_numbers[1]+'), cuda (v '+cuda_version+'). The training was accelerated using a '+gpu_name+'GPU.'

  pdf.set_font('')
  pdf.set_font_size(10.)
  pdf.multi_cell(190, 5, txt = text, align='L')
  pdf.ln(1)
  pdf.set_font('')
  pdf.set_font('Arial', size = 10, style = 'B')
  pdf.cell(28, 5, txt='Augmentation: ', ln=0)
  pdf.set_font('')
  if augmentation:
    aug_text = 'The dataset was augmented by'
    if add_gaussian_blur == True:
      aug_text = aug_text+'\n- gaussian blur'
    if add_linear_contrast == True:
      aug_text = aug_text+'\n- linear contrast'
    if add_additive_gaussian_noise == True:
      aug_text = aug_text+'\n- additive gaussian noise'
    if augmenters != '':
      aug_text = aug_text+'\n- imgaug augmentations: '+augmenters
    if add_elastic_deform == True:
      aug_text = aug_text+'\n- elastic deformation'
  else:
    aug_text = 'No augmentation was used for training.'
  pdf.multi_cell(190, 5, txt=aug_text, align='L')
  pdf.ln(1)
  pdf.set_font('Arial', size = 11, style = 'B')
  pdf.cell(180, 5, txt = 'Parameters', align='L', ln=1)
  pdf.set_font('')
  pdf.set_font_size(10.)
  if use_default_advanced_parameters:
    pdf.cell(200, 5, txt='Default Advanced Parameters were enabled')
  pdf.cell(200, 5, txt='The following parameters were used for training:')
  pdf.ln(1)
  html = """
  <table style="margin-left:0px;">
    <tr>
      <th align="left">Parameter</th>
      <th align="left">Value</th>
    </tr>
    <tr>
      <td>number_of_epochs</td>
      <td>{0}</td>
    </tr>
    <tr>
      <td>batch_size</td>
      <td>{1}</td>
    </tr>
    <tr>
      <td>patch_size</td>
      <td>{2}</td>
    </tr>
    <tr>
      <td>image_pre_processing</td>
      <td>{3}</td>
    </tr>
    <tr>
      <td>validation_split_in_percent</td>
      <td>{4}</td>
    </tr>
      <tr>
      <td>downscaling_in_xy</td>
      <td>{5}</td>
    </tr>
      <tr>
      <td>binary_target</td>
      <td>{6}</td>
    </tr>
    <tr>
      <td>loss_function</td>
      <td>{7}</td>
    </tr>
    <tr>
      <td>metrics</td>
      <td>{8}</td>
    </tr>
    <tr>
      <td>optimizer</td>
      <td>{9}</td>
    </tr>
    <tr>
      <td>checkpointing_period</td>
      <td>{10}</td>
    </tr>
    <tr>
      <td>save_best_only</td>
      <td>{11}</td>
    </tr>
    <tr>
      <td>resume_training</td>
      <td>{12}</td>
    </tr>
  </table>
  """.format(number_of_epochs,batch_size,str(patch_size[0])+'x'+str(patch_size[1])+'x'+str(patch_size[2]),image_pre_processing, validation_split_in_percent, downscaling_in_xy, str(binary_target), loss_function, metrics, optimizer, checkpointing_period, str(save_best_only), str(resume_training))
  pdf.write_html(html)

  #pdf.multi_cell(190, 5, txt = text_2, align='L')
  pdf.set_font("Arial", size = 11, style='B')
  pdf.ln(1)
  pdf.cell(190, 5, txt = 'Training Dataset', align='L', ln=1)
  pdf.set_font('')
  pdf.set_font('Arial', size = 10, style = 'B')
  pdf.cell(30, 5, txt= 'Training_source:', align = 'L', ln=0)
  pdf.set_font('')
  pdf.multi_cell(170, 5, txt = training_source, align = 'L')
  pdf.ln(1)
  pdf.set_font('')
  pdf.set_font('Arial', size = 10, style = 'B')
  pdf.cell(28, 5, txt= 'Training_target:', align = 'L', ln=0)
  pdf.set_font('')
  pdf.multi_cell(170, 5, txt = training_target, align = 'L')
  pdf.ln(1)
  pdf.set_font('')
  pdf.set_font('Arial', size = 10, style = 'B')
  pdf.cell(21, 5, txt= 'Model Path:', align = 'L', ln=0)
  pdf.set_font('')
  pdf.multi_cell(170, 5, txt = model_path+'/'+model_name, align = 'L')
  pdf.ln(1)
  pdf.cell(60, 5, txt = 'Example Training pair (single slice)', ln=1)
  pdf.ln(1)
  exp_size = io.imread(base_path + '/TrainingDataExample_Unet3D.png').shape
  pdf.image(base_path + '/TrainingDataExample_Unet3D.png', x = 11, y = None, w = round(exp_size[1]/8), h = round(exp_size[0]/8))
  pdf.ln(1)
  ref_1 = 'References:\n - ZeroCostDL4Mic: von Chamier, Lucas & Laine, Romain, et al. "Democratising deep learning for microscopy with ZeroCostDL4Mic." Nature Communications (2021).'
  pdf.multi_cell(190, 5, txt = ref_1, align='L')
  pdf.ln(1)
  ref_2 = '- Unet 3D: Çiçek, Özgün, et al. "3D U-Net: learning dense volumetric segmentation from sparse annotation." International conference on medical image computing and computer-assisted intervention. Springer, Cham, 2016.'
  pdf.multi_cell(190, 5, txt = ref_2, align='L')
  # if Use_Data_augmentation:
  #   ref_4 = '- Augmentor: Bloice, Marcus D., Christof Stocker, and Andreas Holzinger. "Augmentor: an image augmentation library for machine learning." arXiv preprint arXiv:1708.04680 (2017).'
  #   pdf.multi_cell(190, 5, txt = ref_4, align='L')
  pdf.ln(3)
  reminder = 'Important:\nRemember to perform the quality control step on all newly trained models\nPlease consider depositing your training dataset on Zenodo'
  pdf.set_font('Arial', size = 11, style='B')
  pdf.multi_cell(190, 5, txt=reminder, align='C')
  pdf.ln(1)

  pdf.output(model_path+'/'+model_name+'/'+model_name+'_training_report.pdf')

  print('------------------------------')
  print('PDF report exported in '+model_path+'/'+model_name+'/')


def qc_pdf_export():
  class MyFPDF(FPDF, HTMLMixin):
    pass

  pdf = MyFPDF()
  pdf.add_page()
  pdf.set_right_margin(-1)
  pdf.set_font("Arial", size = 11, style='B')

  Network = 'U-Net 3D'

  day = datetime.now()
  datetime_str = str(day)[0:10]

  Header = 'Quality Control report for '+Network+' model ('+QC_model_name+')\nDate: '+datetime_str
  pdf.multi_cell(180, 5, txt = Header, align = 'L')
  pdf.ln(1)

  all_packages = ''
  for requirement in freeze(local_only=True):
    all_packages = all_packages+requirement+', '

  pdf.set_font('')
  pdf.set_font('Arial', size = 11, style = 'B')
  pdf.ln(2)
  pdf.cell(190, 5, txt = 'Loss curves', ln=1, align='L')
  pdf.ln(1)
  if os.path.exists(os.path.join(full_QC_model_path,'Quality Control')+'/lossCurvePlots.png'):
    exp_size = io.imread(os.path.join(full_QC_model_path,'Quality Control')+'/lossCurvePlots.png').shape
    pdf.image(os.path.join(full_QC_model_path,'Quality Control')+'/lossCurvePlots.png', x = 11, y = None, w = round(exp_size[1]/8), h = round(exp_size[0]/8))
  else:
    pdf.set_font('')
    pdf.set_font('Arial', size=10)
    pdf.multi_cell(190, 5, txt='If you would like to see the evolution of the loss function during training please play the first cell of the QC section in the notebook.')
  pdf.ln(2)
  pdf.set_font('')
  pdf.set_font('Arial', size = 10, style = 'B')
  pdf.ln(3)
  pdf.cell(80, 5, txt = 'Example Quality Control Visualisation', ln=1)
  pdf.ln(1)
  exp_size = io.imread(os.path.join(full_QC_model_path,'Quality Control')+'/QC_example_data.png').shape
  pdf.image(os.path.join(full_QC_model_path,'Quality Control')+'/QC_example_data.png', x = 5, y = None, w = round(exp_size[1]/12), h = round(exp_size[0]/8))
  pdf.ln(1)
  pdf.set_font('')
  pdf.set_font('Arial', size = 11, style = 'B')
  pdf.ln(1)
  pdf.cell(180, 5, txt = 'IoU threshold optimisation', align='L', ln=1)
  pdf.set_font('')
  pdf.set_font_size(10.)
  pdf.ln(1)
  pdf.cell(120, 5, txt='Highest IoU is {:.4f} with a threshold of {}'.format(best_iou, best_thresh), align='L', ln=1)
  pdf.ln(2)
  exp_size = io.imread(os.path.join(full_QC_model_path,'Quality Control')+'/QC_IoU_analysis.png').shape
  pdf.image(os.path.join(full_QC_model_path,'Quality Control')+'/QC_IoU_analysis.png', x=16, y=None, w = round(exp_size[1]/6), h = round(exp_size[0]/6))
  pdf.ln(1)
  pdf.set_font('')
  pdf.set_font_size(10.)
  ref_1 = 'References:\n - ZeroCostDL4Mic: von Chamier, Lucas & Laine, Romain, et al. "Democratising deep learning for microscopy with ZeroCostDL4Mic." Nature Communications (2021).'
  pdf.multi_cell(190, 5, txt = ref_1, align='L')
  pdf.ln(1)
  ref_2 = '- Unet 3D: Çiçek, Özgün, et al. "3D U-Net: learning dense volumetric segmentation from sparse annotation." International conference on medical image computing and computer-assisted intervention. Springer, Cham, 2016.'
  pdf.multi_cell(190, 5, txt = ref_2, align='L')
  pdf.ln(1)

  pdf.ln(3)
  reminder = 'To find the parameters and other information about how this model was trained, go to the training_report.pdf of this model which should be in the folder of the same name.'

  pdf.set_font('Arial', size = 11, style='B')
  pdf.multi_cell(190, 5, txt=reminder, align='C')
  pdf.ln(1)

  pdf.output(os.path.join(full_QC_model_path,'Quality Control')+'/'+QC_model_name+'_QC_report.pdf')

  print('------------------------------')
  print('QC PDF report exported in '+os.path.join(full_QC_model_path,'Quality Control')+'/')


# -------------- Other definitions -----------
W  = '\033[0m'  # white (normal)
R  = '\033[31m' # red
prediction_prefix = 'Predicted_'


print('-------------------')
print('U-Net 3D and dependencies installed.')

# Colors for the warning messages
class bcolors:
  WARNING = '\033[31m'
  NORMAL = '\033[0m'  # white (normal)


# Check if this is the latest version of the notebook
# Latest_notebook_version = pd.read_csv("https://raw.githubusercontent.com/HenriquesLab/ZeroCostDL4Mic/master/Colab_notebooks/Latest_ZeroCostDL4Mic_Release.csv")

# if Notebook_version == list(Latest_notebook_version.columns):
#   print("This notebook is up-to-date.")

# if not Notebook_version == list(Latest_notebook_version.columns):
#   print(bcolors.WARNING +"A new version of this notebook has been released. We recommend that you download it at https://github.com/HenriquesLab/ZeroCostDL4Mic/wiki")

All_notebook_versions = pd.read_csv("https://raw.githubusercontent.com/HenriquesLab/ZeroCostDL4Mic/master/Colab_notebooks/Latest_Notebook_versions.csv", dtype=str)
print('Notebook version: '+Notebook_version)
Latest_Notebook_version = All_notebook_versions[All_notebook_versions["Notebook"] == Network]['Version'].iloc[0]
print('Latest notebook version: '+Latest_Notebook_version)
if Notebook_version == Latest_Notebook_version:
  print("This notebook is up-to-date.")
else:
  print(bcolors.WARNING +"A new version of this notebook has been released. We recommend that you download it at https://github.com/HenriquesLab/ZeroCostDL4Mic/wiki")


# Build requirements file for local run
after = [str(m) for m in sys.modules]
build_requirements_file(before, after)
!pip3 freeze > requirements.txt

# **2. Complete the Colab session**




---







## **2.1. Check for GPU access**
---

By default, the session should be using Python 3 and GPU acceleration, but it is possible to ensure that these are set properly by doing the following:

<font size = 4>Go to **Runtime -> Change the Runtime type**

<font size = 4>**Runtime type: Python 3** *(Python 3 is programming language in which this program is written)*

<font size = 4>**Accelerator: GPU** *(Graphics processing unit)*


In [ ]:
#@markdown ##Run this cell to check if you have GPU access

!if type nvidia-smi >/dev/null 2>&1; then \
    echo "You have GPU access"; nvidia-smi; \
    else \
    echo -e "You do not have GPU access.\nDid you change your runtime?\nIf the runtime setting is correct then Google did not allocate a GPU for your session\nExpect slow performance. To access GPU try reconnecting later"; fi

## **2.2. Mount Google Drive**
---
<font size = 4> To use this notebook with your **own data**, place it in a folder on **Google Drive** following one of the directory structures outlined in **Section 0**.

1. <font size = 4> **Run** the **cell** below to mount your Google Drive and follow the link.

2. <font size = 4>**Sign in** to your Google account and press 'Allow'.

3. <font size = 4>Next, copy the **authorization code**, paste it into the cell and press enter. This will allow Colab to read and write data from and to your Google Drive.

4. <font size = 4> Once this is done, your data can be viewed in the **Files tab** on the top left of the notebook after hitting 'Refresh'.

In [ ]:
#@markdown ##Play the cell to connect your Google Drive to Colab

# mount user's Google Drive to Google Colab.
from google.colab import drive
drive.mount('/content/gdrive')

# **3. Select your parameters and paths**
---

## **3.1. Choosing parameters**

---

### **Paths to training data and model**

* <font size = 4>**`training_source`** and **`training_target`** specify the paths to the training data. They can either be a single multipage TIFF file each or directories containing various multipage TIFF files in which case target and source files must be named identically within the respective directories. See Section 0 for a detailed description of the necessary directory structure.

* <font size = 4>**`model_name`** will be used when naming checkpoints. Adhere to a `lower_case_with_underscores` naming convention and beware of using the name of an existing model within the same folder, as it will be overwritten.

* <font size = 4>**`model_path`** specifies the directory where the model checkpoints and quality control logs will be saved.


<font size = 4>**Note:** You can copy paths from the 'Files' tab by right-clicking any folder or file and selecting 'Copy path'.

### **Training parameters**

* <font size = 4>**`number_of_epochs`** is the number of times the entire training data will be seen by the model. *Default: >100*

* <font size = 4>**`batch_size`** is the number of training patches of size `patch_size` that will be bundled together at each training step. *Default: 1*

* <font size = 4>**`patch_size`** specifies the size of the three-dimensional training patches in (x, y, z) that will be fed to the model. In order to avoid errors, preferably use a square aspect ratio or stick to the advanced parameters. *Default: <(512, 512, 16)*

* <font size = 4>**`validation_split_in_percent`** is the relative amount of training data that will be set aside for validation. *Default: 20*

* <font size = 4>**`downscaling_in_xy`** downscales the training images by the specified amount in x and y. This is useful to enforce isotropic pixel-size if the z resolution is lower than the xy resolution in the training volume or to capture a larger field-of-view while decreasing the memory requirements. *Default: 1*

* <font size = 4>**`image_pre_processing`** selects whether the training images are randomly cropped during training or resized to `patch_size`. Choose `randomly crop to patch_size` to shrink the field-of-view of the training images to the `patch_size`. *Default: resize to patch_size*

* <font size = 4>**`binary_target`** forces the target image to be binary. Choose this if your model is trained to perform binary segmentation tasks *Default: True*

* <font size = 4>**`loss_function`** defines the loss. Read more [here](https://keras.io/api/losses/). *Default: weighted_binary_crossentropy*

* <font size = 4>**`metrics`** defines the metric. Read more [here](https://keras.io/api/metrics/). *Default: dice*

* <font size = 4>**`optimizer`** defines the optimizer. Read more [here](https://keras.io/api/optimizers/). *Default: adam*

<font size = 4>**Note:** If a *ResourceExhaustedError* is raised in Section 4.1. during training, decrease `batch_size` and `patch_size`. Decrease `batch_size` first and if the error persists at `batch_size = 1`, reduce the `patch_size`.  

<font size = 4>**Note:** The number of steps per epoch are calculated as `floor(augment_factor * (1 - validation_split) * num_of_slices / batch_size)` if `image_pre_processing` is `resize to patch_size` where `augment_factor` is three if `apply_data_augmentation` is `True` and one otherwise. The `num_of_slices` is the overall number of slices (z-depth) in the training set across all provided image volumes. If `image_pre_processing` is `randomly crop to patch_size`, the number of steps per epoch are calculated as `floor(augment_factor * volume / (crop_volume * batch_size))` where `volume` is the overall volume of the training data in pixels accounting for the validation split and `crop_volume` is defined as the volume in pixels based on the specified `patch_size`.

In [ ]:
#@markdown ###Path to training data:
training_source = "" #@param {type:"string"}
# Check that the base_path is on the path and otherwise add it
training_source = check_base_path(base_path, training_source)

training_target = "" #@param {type:"string"}
# Check that the base_path is on the path and otherwise add it
training_target = check_base_path(base_path, training_target)

#@markdown ---

#@markdown ###Model name and path to model folder:
model_name = "" #@param {type:"string"}
model_path = "" #@param {type:"string"}

full_model_path = os.path.join(model_path, model_name)
# Check that the base_path is on the path and otherwise add it
full_model_path = check_base_path(base_path, full_model_path)

#@markdown ---

#@markdown ###Training parameters
number_of_epochs =   2#@param {type:"number"}

#@markdown ###Default advanced parameters
use_default_advanced_parameters = False #@param {type:"boolean"}

#@markdown <font size = 3>If not, please change:

batch_size =  1#@param {type:"number"}
patch_size = (512,512,8) #@param {type:"number"} # in pixels
training_shape = patch_size + (1,)
image_pre_processing = 'resize to patch_size' #@param ["randomly crop to patch_size", "resize to patch_size"]

validation_split_in_percent = 20 #@param{type:"number"}
downscaling_in_xy =  1#@param {type:"number"} # in pixels

binary_target = True #@param {type:"boolean"}

loss_function = 'weighted_binary_crossentropy' #@param ["weighted_binary_crossentropy", "binary_crossentropy", "categorical_crossentropy", "sparse_categorical_crossentropy", "mean_squared_error", "mean_absolute_error"]

metrics = 'dice' #@param ["dice", "accuracy"]

optimizer = 'adam' #@param ["adam", "sgd", "rmsprop"]

learning_rate = 0.001 #@param{type:"number"}

if image_pre_processing == "randomly crop to patch_size":
    random_crop = True
else:
    random_crop = False

if use_default_advanced_parameters:
    print("Default advanced parameters enabled")
    batch_size = 3
    training_shape = (256,256,8,1)
    validation_split_in_percent = 20
    downscaling_in_xy = 1
    random_crop = True
    binary_target = True
    loss_function = 'weighted_binary_crossentropy'
    metrics = 'dice'
    optimizer = 'adam'
    learning_rate = 0.001

#@markdown ###Checkpointing parameters
#checkpointing_period = 1 #@param {type:"number"}
checkpointing_period = "epoch"
#@markdown  <font size = 3>If chosen, only the best checkpoint is saved. Otherwise a checkpoint is saved every epoch:
save_best_only = False #@param {type:"boolean"}

#@markdown ###Resume training
#@markdown <font size = 3>Choose if training was interrupted:
resume_training = False #@param {type:"boolean"}

#@markdown ###Transfer learning
#@markdown <font size = 3>For transfer learning, do not select resume_training and specify a checkpoint_path below.
#@markdown <font size = 3> - If the model is already downloaded or is locally available, please specify the path to the .h5 file.
#@markdown <font size = 3> - To use a model from the BioImage Model Zoo, write the model ID. For example: 10.5281/zenodo.5749843

pretrained_model_choice = "Model_from_file" #@param ["Model_from_file", "bioimageio_model"]
checkpoint_path = "" #@param {type:"string"}
model_id = "" #@param {type:"string"}
# --------------------- Load the model from a bioimageio model (can be path on drive or url / doi) ---
if pretrained_model_choice == "bioimageio_model":
  model_spec = load_raw_resource_description(model_id)
  if "keras_hdf5" not in model_spec.weights:
    print("Invalid bioimageio model")
  else:
    url = model_spec.weights["keras_hdf5"].source
    r = requests.get(url, allow_redirects=True)
    open("keras_model.h5", 'wb').write(r.content)
    checkpoint_path = "keras_model.h5"

if resume_training and checkpoint_path != "":
    print('If resume_training is True while checkpoint_path is specified, resume_training will be set to False!')
    resume_training = False

# Retrieve last checkpoint
if resume_training:
    try:
      ckpt_dir_list = glob(full_model_path + '/ckpt/*')
      ckpt_dir_list.sort()
      last_ckpt_path = ckpt_dir_list[-1]
      print('Training will resume from checkpoint:', os.path.basename(last_ckpt_path))
    except IndexError:
      last_ckpt_path=None
      print('CheckpointError: No previous checkpoints were found, training from scratch.')
elif not resume_training and checkpoint_path != "":
    last_ckpt_path = checkpoint_path
    assert os.path.isfile(last_ckpt_path), 'checkpoint_path does not exist!'
else:
    last_ckpt_path=None

# Instantiate Unet3D
model = Unet3D(shape=training_shape)

#here we check that no model with the same name already exist
if not resume_training and os.path.exists(full_model_path):
    print(bcolors.WARNING+'The model folder already exists and will be overwritten.'+bcolors.NORMAL)
    # print('!! WARNING: Folder already exists and will be overwritten !!')
    # shutil.rmtree(full_model_path)

# if not os.path.exists(full_model_path):
#     os.makedirs(full_model_path)

# Show sample image
if os.path.isdir(training_source):
    training_source_sample = sorted(glob(os.path.join(training_source, '*')))[0]
    training_target_sample = sorted(glob(os.path.join(training_target, '*')))[0]
else:
    training_source_sample = training_source
    training_target_sample = training_target

src_sample = tifffile.imread(training_source_sample)
src_sample = model._min_max_scaling(src_sample)
if binary_target:
    tgt_sample = tifffile.imread(training_target_sample).astype(bool)
else:
    tgt_sample = tifffile.imread(training_target_sample)

src_down = transform.downscale_local_mean(src_sample[0], (downscaling_in_xy, downscaling_in_xy))
tgt_down = transform.downscale_local_mean(tgt_sample[0], (downscaling_in_xy, downscaling_in_xy))

if random_crop:
    true_patch_size = None

    if src_down.shape[0] == training_shape[0]:
      x_rand = 0
    if src_down.shape[1] == training_shape[1]:
      y_rand = 0
    if src_down.shape[0] > training_shape[0]:
      x_rand = np.random.randint(src_down.shape[0] - training_shape[0])
    if src_down.shape[1] > training_shape[1]:
      y_rand = np.random.randint(src_down.shape[1] - training_shape[1])
    if src_down.shape[0] < training_shape[0] or src_down.shape[1] < training_shape[1]:
      raise ValueError('Patch shape larger than (downscaled) source shape')
else:
    true_patch_size = src_down.shape

def scroll_in_z(z):
    src_down = transform.downscale_local_mean(src_sample[z-1], (downscaling_in_xy,downscaling_in_xy))
    tgt_down = transform.downscale_local_mean(tgt_sample[z-1], (downscaling_in_xy,downscaling_in_xy))
    if random_crop:
        src_slice = src_down[x_rand:training_shape[0]+x_rand, y_rand:training_shape[1]+y_rand]
        tgt_slice = tgt_down[x_rand:training_shape[0]+x_rand, y_rand:training_shape[1]+y_rand]
    else:

        src_slice = transform.resize(src_down, (training_shape[0], training_shape[1]), mode='constant', preserve_range=True)
        tgt_slice = transform.resize(tgt_down, (training_shape[0], training_shape[1]), mode='constant', preserve_range=True)

    f=plt.figure(figsize=(16,8))
    plt.subplot(1,2,1)
    plt.imshow(src_slice, cmap='gray')
    plt.title('Training source (z = ' + str(z) + ')', fontsize=15)
    plt.axis('off')

    plt.subplot(1,2,2)
    plt.imshow(tgt_slice, cmap='magma')
    plt.title('Training target (z = ' + str(z) + ')', fontsize=15)
    plt.axis('off')
    plt.savefig(base_path + '/TrainingDataExample_Unet3D.png',bbox_inches='tight',pad_inches=0)
    #plt.close()

print('This is what the training images will look like with the chosen settings')
interact(scroll_in_z, z=widgets.IntSlider(min=1, max=src_sample.shape[0], step=1, value=0));
plt.show()
#Create a copy of an example slice and close the display.
scroll_in_z(z=int(src_sample.shape[0]/2))
# If you close the display, then the users can't interactively inspect the data
# plt.close()

# Save model parameters
params =  {'training_source': training_source,
           'training_target': training_target,
           'model_name': model_name,
           'model_path': model_path,
           'number_of_epochs': number_of_epochs,
           'batch_size': batch_size,
           'training_shape': training_shape,
           'downscaling': downscaling_in_xy,
           'true_patch_size': true_patch_size,
           'val_split': validation_split_in_percent/100,
           'random_crop': random_crop}

params_df = pd.DataFrame.from_dict(params, orient='index')

# apply_data_augmentation = False
# pdf_export(augmentation = apply_data_augmentation, pretrained_model = resume_training)

## **3.2. Data augmentation**

---
<font size = 4> Augmenting the training data increases robustness of the model by simulating possible variations within the training data which avoids it from overfitting on small datasets. We therefore strongly recommended augmenting the data and making sure that the applied augmentations are reasonable.

* <font size = 4>**Gaussian blur** blurs images using Gaussian kernels with a sigma of `gaussian_sigma`. This augmentation step is applied with a probability of `gaussian_frequency`. Read more [here](https://imgaug.readthedocs.io/en/latest/source/overview/blur.html#gaussianblur).

* <font size = 4>**Linear contrast** modifies the contrast of images according to `127 + alpha *(pixel_value-127)`, where `pixel_value` and `alpha` are sampled uniformly from the interval `[contrast_min, contrast_max]`. This augmentation step is applied with a probability of `contrast_frequency`. Read more [here](https://imgaug.readthedocs.io/en/latest/source/overview/contrast.html#linearcontrast).

* <font size = 4>**Additive Gaussian noise** adds Gaussian noise sampled once per pixel from a normal distribution `N(0, s)`, where `s` is sampled from `[scale_min, scale_max]`. This augmentation step is applied with a probability of `noise_frequency`. Read more [here](https://imgaug.readthedocs.io/en/latest/source/overview/arithmetic.html#additivegaussiannoise).

* <font size = 4>**Add custom augmenters** allows you to create a custom augmentation pipeline using the [augmenters available in the imagug library](https://imgaug.readthedocs.io/en/latest/source/overview_of_augmenters.html).
In the example above, the augmentation pipeline is equivalent to:
```
seq = iaa.Sequential([
    iaa.Sometimes(0.3, iaa.GammaContrast((0.5, 2.0)),
    iaa.Sometimes(0.4, iaa.AverageBlur((0.5, 2.0)),
    iaa.Sometimes(0.5, iaa.LinearContrast((0.4, 1.6)),
], random_order=True)
```
<font size = 4> Note that there is no limit on the number of augmenters that can be chained together and that individual augmenter and parameter entries must be separated by `;`. Custom augmenters do not overwrite the preset augmentation steps (*Gaussian blur*, *Linear contrast* or *Additive Gaussian noise*). Also, the augmenters, augmenter parameters and augmenter frequencies must be entered such that each position within the string corresponds to the same augmentation step.

* <font size = 4>**`apply_data_augmentation`** ensures that data augmentation is randomly applied to the training data at each training step. This includes inverting the order of the slices within a training patch, as well as applying any augmenters that are added. *Default: True*

* <font size = 4>**`add_elastic_deform`** ensures that elastic grid-based deformations are applied as described in the original 3D U-Net paper. *Default: True*

In [ ]:
#@markdown ##**Augmentation options**

#@markdown ###Data augmentation

apply_data_augmentation = True #@param {type:"boolean"}

# List of augmentations
augmentations = []

#@markdown ###Gaussian blur
add_gaussian_blur = True #@param {type:"boolean"}
gaussian_sigma =   0.7#@param {type:"number"}
gaussian_frequency = 0.5 #@param {type:"number"}

if add_gaussian_blur:
    augmentations.append(iaa.Sometimes(gaussian_frequency, iaa.GaussianBlur(sigma=(0, gaussian_sigma))))

#@markdown ###Linear contrast
add_linear_contrast = True #@param {type:"boolean"}
contrast_min =  0.4 #@param {type:"number"}
contrast_max =   1.6#@param {type:"number"}
contrast_frequency = 0.5 #@param {type:"number"}

if add_linear_contrast:
    augmentations.append(iaa.Sometimes(contrast_frequency, iaa.LinearContrast((contrast_min, contrast_max))))

#@markdown ###Additive Gaussian noise
add_additive_gaussian_noise = True #@param {type:"boolean"}
scale_min =  0 #@param {type:"number"}
scale_max =  0.05 #@param {type:"number"}
noise_frequency = 0.5 #@param {type:"number"}

if add_additive_gaussian_noise:
    augmentations.append(iaa.Sometimes(noise_frequency, iaa.AdditiveGaussianNoise(scale=(scale_min, scale_max))))

#@markdown ###Add custom augmenters
add_custom_augmenters = False #@param {type:"boolean"}
augmenters = "" #@param {type:"string"}

if add_custom_augmenters:

    augmenter_params = "" #@param {type:"string"}

    augmenter_frequency = "" #@param {type:"string"}

    aug_lst = augmenters.split(';')
    aug_params_lst = augmenter_params.split(';')
    aug_freq_lst = augmenter_frequency.split(';')

    assert len(aug_lst) == len(aug_params_lst) and len(aug_lst) == len(aug_freq_lst), 'The number of arguments in augmenters, augmenter_params and augmenter_frequency are not the same!'

    for __, (aug, param, freq) in enumerate(zip(aug_lst, aug_params_lst, aug_freq_lst)):
        aug, param, freq = aug.strip(), param.strip(), freq.strip()
        aug_func = iaa.Sometimes(eval(freq), getattr(iaa, aug)(eval(param)))
        augmentations.append(aug_func)

#@markdown ###Elastic deformations
add_elastic_deform = False #@param {type:"boolean"}
sigma =  2#@param {type:"number"}
points =  2#@param {type:"number"}
order =  1#@param {type:"number"}

if add_elastic_deform:
    deform_params = (sigma, points, order)
else:
    deform_params = None

train_generator = MultiPageTiffGenerator(training_source,
                                         training_target,
                                         batch_size=batch_size,
                                         shape=training_shape,
                                         augment=apply_data_augmentation,
                                         augmentations=augmentations,
                                         deform_augment=add_elastic_deform,
                                         deform_augmentation_params=deform_params,
                                         val_split=validation_split_in_percent/100,
                                         random_crop=random_crop,
                                         downscale=downscaling_in_xy,
                                         binary_target=binary_target)

val_generator = MultiPageTiffGenerator(training_source,
                                       training_target,
                                       batch_size=batch_size,
                                       shape=training_shape,
                                       val_split=validation_split_in_percent/100,
                                       is_val=True,
                                       random_crop=random_crop,
                                       downscale=downscaling_in_xy,
                                       binary_target=binary_target)


if apply_data_augmentation:
  print('Data augmentation enabled.')
  sample_src_aug, sample_tgt_aug = train_generator.sample_augmentation(random.randint(0, len(train_generator)))

  def scroll_in_z(z):
      f=plt.figure(figsize=(16,8))
      plt.subplot(1,2,1)
      plt.imshow(sample_src_aug[0,:,:,z-1,0], cmap='gray')
      plt.title('Sample augmented source (z = ' + str(z) + ')', fontsize=15)
      plt.axis('off')

      plt.subplot(1,2,2)
      plt.imshow(sample_tgt_aug[0,:,:,z-1,0], cmap='magma')
      plt.title('Sample training target (z = ' + str(z) + ')', fontsize=15)
      plt.axis('off')

  print('This is what the augmented training images will look like with the chosen settings')
  interact(scroll_in_z, z=widgets.IntSlider(min=1, max=sample_src_aug.shape[3], step=1, value=0));

else:
  print('Data augmentation disabled.')


# **4. Train the network**
---

<font size = 4>**CRITICAL NOTE:** Google Colab has a time limit for processing (to prevent using GPU power for datamining). Training times must be less than 12 hours! If training takes longer than 12 hours, please decrease `number_of_epochs`.

## **4.1. Start Training**
---


In [ ]:
#@markdown ##Start training

model.summary()

#here we check that no model with the same name already exist, if so delete
if not resume_training and os.path.exists(full_model_path):
    shutil.rmtree(full_model_path)
    print(bcolors.WARNING+'!! WARNING: Folder already exists and has been overwritten !!'+bcolors.NORMAL)

if not os.path.exists(full_model_path):
    os.makedirs(full_model_path)

pdf_export(augmentation = apply_data_augmentation, pretrained_model = resume_training)

# Save file
params_df.to_csv(os.path.join(full_model_path, 'params.csv'))

start = time.time()
# Start Training
model.train(epochs=number_of_epochs,
            batch_size=batch_size,
            train_generator=train_generator,
            val_generator=val_generator,
            model_path=model_path,
            model_name=model_name,
            loss=loss_function,
            metrics=metrics,
            optimizer=optimizer,
            learning_rate=learning_rate,
            ckpt_period=checkpointing_period,
            save_best_ckpt_only=save_best_only,
            ckpt_path=last_ckpt_path)

print('Training successfully completed!')
dt = time.time() - start
mins, sec = divmod(dt, 60)
hour, mins = divmod(mins, 60)
print("Time elapsed:",hour, "hour(s)",mins,"min(s)",round(sec),"sec(s)")

#Create a pdf document with training summary

pdf_export(trained = True, augmentation = apply_data_augmentation, pretrained_model = resume_training)

## **4.2. Download your model from Google Drive**

---
<font size = 4>Once training is complete, the trained model is automatically saved to your Google Drive, in the **`model_path`** folder that was specified in Section 3. Download the folder to avoid any unwanted surprises, since the data can be erased if you train another model using the same `model_path`.

In [ ]:
#@markdown ##Download model directory
#@markdown 1.  <font size = 4>Specify the model_path in `model_path_download`. Otherwise, the model that was sepcified in Section 3.1 will be downloaded.

model_path_download = "" #@param {type:"string"}

#@markdown 2.  <font size = 4>Run this cell to zip the model directory and automatically download it on your computer.

if len(model_path_download) == 0:
    model_path_download = full_model_path

model_name_download = os.path.basename(model_path_download)
# Check that the base_path is on the path and otherwise add it
model_name_download = check_base_path(base_path, model_name_download)

print(f"Zipping {model_name_download} ...")

zip_model_path = model_name_download + '.zip'

!zip -q -r "$zip_model_path" "$model_path_download"

print(f"Successfully saved zipped model directory as: {zip_model_path}")

print("The zipped model will be automatically downloaded into your computer:")

from google.colab import files
files.download(zip_model_path)


# **5. Evaluate your model**
---

<font size = 4>In this section the newly trained model can be assessed for performance. This involves inspecting the loss function in Section 5.1. and employing more advanced metrics in Section 5.2.

<font size = 4>**We highly recommend performing quality control on all newly trained models.**



In [ ]:
#@markdown ###Model to be evaluated:
#@markdown ###Do you want to assess the model you just trained ?
Use_the_current_trained_model = True #@param {type:"boolean"}

#@markdown ###If not, please provide the path to the model folder:
# model name and path
#@markdown ###Name of the model and path to model folder:
QC_model_folder = "" #@param {type:"string"}

# In case the user chooses to use the trained model, load previous values
if (Use_the_current_trained_model):
    QC_model_name = model_name
    QC_model_path = model_path
    QC_model_folder = os.path.join(model_path, model_name)

# Check that the base_path is on the path and otherwise add it
QC_model_folder = check_base_path(base_path, QC_model_folder)

#Here we define the loaded model name and path
QC_model_name = os.path.basename(QC_model_folder)
QC_model_path = os.path.dirname(QC_model_folder)

full_QC_model_path =  os.path.join(QC_model_path, QC_model_name)

if os.path.exists(full_QC_model_path):
    print(QC_model_name + ' will be evaluated')
else:
    print(R+'!! WARNING: The chosen model does not exist !!'+W)
    print('Please make sure you provide a valid model path and model name before proceeding further.')


## **5.1. Inspecting loss function**
---

<font size = 4>**The training loss** is the error between prediction and target after each epoch calculated across the training data while the **validation loss** calculates the error on the (unseen) validation data. During training these values should decrease until converging at which point the model has been sufficiently trained. If the validation loss starts increasing while the training loss has plateaued, the model has overfit on the training data which reduces its ability to generalise. Aim to halt training before this point.

<font size = 4>**Note:** For a more in-depth explanation please refer to [this review](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6381354/) by Nichols et al.


<font size = 4>The accuracy is another performance metric that is calculated after each epoch. We use the [Sørensen–Dice coefficient](https://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient) to score the prediction accuracy.



In [ ]:
#@markdown ##Visualise loss and accuracy
lossDataFromCSV = []
vallossDataFromCSV = []
accuracyDataFromCSV = []
valaccuracyDataFromCSV = []

with open(full_QC_model_path + '/Quality Control/training_evaluation.csv', 'r') as csvfile:
    csvRead = csv.reader(csvfile, delimiter=',')
    next(csvRead)
    for row in csvRead:
        lossDataFromCSV.append(float(row[2]))
        vallossDataFromCSV.append(float(row[4]))
        accuracyDataFromCSV.append(float(row[1]))
        valaccuracyDataFromCSV.append(float(row[3]))

epochNumber = range(len(lossDataFromCSV))
plt.figure(figsize=(15,10))

plt.subplot(2,1,1)
plt.plot(epochNumber,lossDataFromCSV, label='Training loss')
plt.plot(epochNumber,vallossDataFromCSV, label='Validation loss')
plt.title('Training and validation loss', fontsize=14)
plt.ylabel('Loss', fontsize=12)
plt.xlabel('Epochs', fontsize=12)
plt.legend()

plt.subplot(2,1,2)
plt.plot(epochNumber,accuracyDataFromCSV, label='Training accuracy')
plt.plot(epochNumber,valaccuracyDataFromCSV, label='Validation accuracy')
plt.title('Training and validation accuracy', fontsize=14)
plt.ylabel('Dice', fontsize=12)
plt.xlabel('Epochs', fontsize=12)
plt.legend()
plt.savefig(full_QC_model_path + '/Quality Control/lossCurvePlots.png', bbox_inches='tight', pad_inches=0)
plt.show()



## **5.2. Error mapping and quality metrics estimation**
---
<font size = 4>This section will provide both a visual indication of the model performance by comparing the overlay of the predicted and source volume.

In [ ]:
#@markdown ##Compare prediction and ground-truth on testing data

#@markdown <font size = 4>Provide an unseen annotated dataset to determine the performance of the model:

testing_source = "" #@param{type:"string"}
# Check that the base_path is on the path and otherwise add it
testing_source = check_base_path(base_path, testing_source)

testing_target = "" #@param{type:"string"}
# Check that the base_path is on the path and otherwise add it
testing_target = check_base_path(base_path, testing_target)

qc_dir = full_QC_model_path + '/Quality Control'
predict_dir = qc_dir + '/Prediction'

if os.path.exists(predict_dir):
    shutil.rmtree(predict_dir)
os.makedirs(predict_dir)

# predict_dir + '/' +
predict_path = os.path.splitext(os.path.basename(testing_source))[0] + '_prediction.tif'

def last_chars(x):
    return(x[-11:])

try:
    ckpt_dir_list = glob(full_QC_model_path + '/ckpt/*')
    ckpt_dir_list.sort(key=last_chars)
    last_ckpt_path = ckpt_dir_list[0]
    print('Predicting from checkpoint:', os.path.basename(last_ckpt_path))
except IndexError:
    raise CheckpointError('No previous checkpoints were found, please retrain model.')

# Load parameters
params = pd.read_csv(os.path.join(full_QC_model_path, 'params.csv'), names=['val'], header=0, index_col=0)

model = Unet3D(shape=params.loc['training_shape', 'val'])

prediction = model.predict(testing_source, last_ckpt_path,
                           downscaling=params.loc['downscaling', 'val'],
                           true_patch_size=params.loc['true_patch_size', 'val'])

tifffile.imwrite(predict_path, prediction.astype('float32'), imagej=True)

print('Predicted images!')

qc_metrics_path = full_QC_model_path + '/Quality Control/QC_metrics_' + QC_model_name + '.csv'

test_target = tifffile.imread(testing_target)
test_source = tifffile.imread(testing_source)
test_prediction = tifffile.imread(predict_path)

def scroll_in_z(z):

    plt.figure(figsize=(25,5))
    # Source
    plt.subplot(1,4,1)
    plt.axis('off')
    plt.imshow(test_source[z-1], cmap='gray')
    plt.title('Source (z = ' + str(z) + ')', fontsize=15)

    # Target (Ground-truth)
    plt.subplot(1,4,2)
    plt.axis('off')
    plt.imshow(test_target[z-1], cmap='magma')
    plt.title('Target (z = ' + str(z) + ')', fontsize=15)

    # Prediction
    plt.subplot(1,4,3)
    plt.axis('off')
    plt.imshow(test_prediction[z-1], cmap='magma')
    plt.title('Prediction (z = ' + str(z) + ')', fontsize=15)

    # Overlay
    plt.subplot(1,4,4)
    plt.axis('off')
    plt.imshow(test_target[z-1], cmap='Greens')
    plt.imshow(test_prediction[z-1], alpha=0.5, cmap='Purples')
    plt.title('Overlay (z = ' + str(z) + ')', fontsize=15)
    plt.savefig(os.path.join(full_QC_model_path,'Quality Control')+'/QC_example_data.png', bbox_inches='tight', pad_inches=0)
interact(scroll_in_z, z=widgets.IntSlider(min=1, max=test_source.shape[0], step=1, value=0));

## **5.3. Determine best Intersection over Union and threshold**
---

<font size = 4>**Note:** This section is only relevant if the target image is a binary mask and `binary_target` is selected in Section 3!  

<font size = 4>This section will provide both a visual and a quantitative indication of the model performance by comparing the overlay of the predicted and source volume, as well as computing the highest [**Intersection over Union**](https://en.wikipedia.org/wiki/Jaccard_index) (IoU) score. The IoU is also known as the Jaccard Index.  

<font size = 4>The best threshold is calculated using the IoU. Each threshold value from 0 to 255 is tested and the threshold with the highest score is deemed the best. The IoU is calculated for the entire volume in 3D.

In [ ]:

#@markdown ##Calculate Intersection over Union and best threshold
prediction = tifffile.imread(predict_path)
prediction = np.interp(prediction, (prediction.min(), prediction.max()), (0, 255))

target = tifffile.imread(testing_target).astype(bool)

def iou_vs_threshold(prediction, target):
    threshold_list = []
    IoU_scores_list = []

    for threshold in range(0,256):
        mask = prediction > threshold

        intersection = np.logical_and(target, mask)
        union = np.logical_or(target, mask)
        iou_score = np.sum(intersection) / np.sum(union)

        threshold_list.append(threshold)
        IoU_scores_list.append(iou_score)

    return threshold_list, IoU_scores_list

threshold_list, IoU_scores_list = iou_vs_threshold(prediction, target)
thresh_arr = np.array(list(zip(threshold_list, IoU_scores_list)))
best_thresh = int(np.where(thresh_arr == np.max(thresh_arr[:,1]))[0])
best_iou = IoU_scores_list[best_thresh]

print('Highest IoU is {:.4f} with a threshold of {}'.format(best_iou, best_thresh))

def adjust_threshold(threshold, z):

    f=plt.figure(figsize=(25,5))
    plt.subplot(1,4,1)
    plt.imshow((prediction[z-1] > threshold).astype('uint8'), cmap='magma')
    plt.title('Prediction (Threshold = ' + str(threshold) + ')', fontsize=15)
    plt.axis('off')

    plt.subplot(1,4,2)
    plt.imshow(target[z-1], cmap='magma')
    plt.title('Target (z = ' + str(z) + ')', fontsize=15)
    plt.axis('off')

    plt.subplot(1,4,3)
    plt.axis('off')
    plt.imshow(test_source[z-1], cmap='gray')
    plt.imshow((prediction[z-1] > threshold).astype('uint8'), alpha=0.4, cmap='Reds')
    plt.title('Overlay (z = ' + str(z) + ')', fontsize=15)

    plt.subplot(1,4,4)
    plt.title('Threshold vs. IoU', fontsize=15)
    plt.plot(threshold_list, IoU_scores_list)
    plt.plot(threshold, IoU_scores_list[threshold], 'ro')
    plt.ylabel('IoU score')
    plt.xlabel('Threshold')
    plt.savefig(os.path.join(full_QC_model_path,'Quality Control')+'/QC_IoU_analysis.png',bbox_inches=matplotlib.transforms.Bbox([[17.5,0],[23,5]]),pad_inches=0)
    plt.show()

interact(adjust_threshold,
         threshold=widgets.IntSlider(min=0, max=255, step=1, value=best_thresh),
         z=widgets.IntSlider(min=1, max=prediction.shape[0], step=1, value=0));

#Make a pdf summary of the QC results

qc_pdf_export()

## **5.4. Export your model into the BioImage Model Zoo format**
---
<font size = 4>This section exports the model into the [BioImage Model Zoo](https://bioimage.io/#/) format so it can be used directly with deepImageJ or Ilastik. The new files will be stored in the model folder specified at the beginning of Section 5.

<font size = 4>Once the cell is executed, you will find a new zip file with the name specified in `trained_model_name.bioimage.io.model`.

<font size = 4>To use it with deepImageJ, download it and install it suing DeepImageJ Install Model > Install from a local file.

<font size = 4>To try the model in ImageJ, go to Plugins > DeepImageJ > DeepImageJ Run, choose this model from the list and click on Test Model.

<font size = 4>  More information at https://deepimagej.github.io/deepimagej/

In [ ]:
# ------------- User input ------------
# information about the model
#@markdown ##Insert the information to document your model:
Trained_model_name    = "" #@param {type:"string"}
assert len(Trained_model_name) > 5, "Please provide a Trained_model_name longer than 5 words."

Trained_model_description = "A 3D U-Net trained using ZeroCostDL4Mic to segment mitochondria in 3D transmission electron microscopy images of the CA1 hippocampus region." #@param {type:"string"}
# Information about the authors, packager, mantainer
# @markdown ## Author(s) names and affiliations, comma separated:
# @markdown > *NOTICE:* The order and number of affiliations need to be the same as the authors
Trained_model_authors = "Author 1 Name, Author 2 Name, Author 3 Name" #@param {type:"string"}
Trained_model_authors_affiliation = "Author 1 Affiliation, Author 2 Affiliation, Author 3 Affiliation" #@param {type:"string"}

# Check that the length of author list and affiliation list are the shame
list_auth_names = Trained_model_authors.replace(" ", "").split(",") # Remove spaces and split based on comas
list_auth_names = [e for e in list_auth_names if e] # Remove empty cases (maybe due to additional final coma)
list_auth_affs = Trained_model_authors_affiliation.replace(" ", "").split(",") # Remove spaces and split based on comas
list_auth_affs = [e for e in list_auth_affs if e] # Remove empty cases (maybe due to additional final coma)
assert len(list_auth_names) == len(list_auth_affs)

# @markdown ## Model Packager:
# @markdown The packager will be taken from the list of authors above. You just need to choose the packager position in the list of authors.
# @markdown > *E.g.* 1 for Author 1, 2 for Author 2, etc.
Trained_model_packager = 1 #@param {type:"number"}

# @markdown ## Model Maintainer:

# @markdown ### Maintainer's name:
# @markdown If maintainers name is **NOT** in author list, please provide here the name:
Trained_model_maintainer_name =  "Maintainer Name" #@param {type:"string"}
# @markdown Otherwise, provide the corresponding position number:
# @markdown > *E.g.* 1 for Author 1, 2 for Author 2, etc.
Trained_model_maintainer_number = 1 #@param {type:"number"}

# @markdown ### Maintainer's GitHub user:
Trained_model_maintainer_github = "" #@param {type:"string"}
assert len(Trained_model_maintainer_github) > 0, "Please provide maintainer's GitHub user name"

# @markdown ###License:
Trained_model_license = "MIT" #@param ["0BSD","AAL","Abstyles","AdaCore-doc","Adobe-2006","Adobe-Display-PostScript","Adobe-Glyph","Adobe-Utopia","ADSL","AFL-1.1","AFL-1.2","AFL-2.0","AFL-2.1","AFL-3.0","Afmparse","AGPL-1.0-only","AGPL-1.0-or-later","AGPL-3.0-only","AGPL-3.0-or-later","Aladdin","AMDPLPA","AML","AML-glslang","AMPAS","ANTLR-PD","ANTLR-PD-fallback","Apache-1.0","Apache-1.1","Apache-2.0","APAFML","APL-1.0","App-s2p","APSL-1.0","APSL-1.1","APSL-1.2","APSL-2.0","Arphic-1999","Artistic-1.0","Artistic-1.0-cl8","Artistic-1.0-Perl","Artistic-2.0","ASWF-Digital-Assets-1.0","ASWF-Digital-Assets-1.1","Baekmuk","Bahyph","Barr","bcrypt-Solar-Designer","Beerware","Bitstream-Charter","Bitstream-Vera","BitTorrent-1.0","BitTorrent-1.1","blessing","BlueOak-1.0.0","Boehm-GC","Borceux","Brian-Gladman-2-Clause","Brian-Gladman-3-Clause","BSD-1-Clause","BSD-2-Clause","BSD-2-Clause-Darwin","BSD-2-Clause-Patent","BSD-2-Clause-Views","BSD-3-Clause","BSD-3-Clause-acpica","BSD-3-Clause-Attribution","BSD-3-Clause-Clear","BSD-3-Clause-flex","BSD-3-Clause-HP","BSD-3-Clause-LBNL","BSD-3-Clause-Modification","BSD-3-Clause-No-Military-License","BSD-3-Clause-No-Nuclear-License","BSD-3-Clause-No-Nuclear-License-2014","BSD-3-Clause-No-Nuclear-Warranty","BSD-3-Clause-Open-MPI","BSD-3-Clause-Sun","BSD-4-Clause","BSD-4-Clause-Shortened","BSD-4-Clause-UC","BSD-4.3RENO","BSD-4.3TAHOE","BSD-Advertising-Acknowledgement","BSD-Attribution-HPND-disclaimer","BSD-Inferno-Nettverk","BSD-Protection","BSD-Source-beginning-file","BSD-Source-Code","BSD-Systemics","BSD-Systemics-W3Works","BSL-1.0","BUSL-1.1","bzip2-1.0.6","C-UDA-1.0","CAL-1.0","CAL-1.0-Combined-Work-Exception","Caldera","Caldera-no-preamble","CATOSL-1.1","CC-BY-1.0","CC-BY-2.0","CC-BY-2.5","CC-BY-2.5-AU","CC-BY-3.0","CC-BY-3.0-AT","CC-BY-3.0-AU","CC-BY-3.0-DE","CC-BY-3.0-IGO","CC-BY-3.0-NL","CC-BY-3.0-US","CC-BY-4.0","CC-BY-NC-1.0","CC-BY-NC-2.0","CC-BY-NC-2.5","CC-BY-NC-3.0","CC-BY-NC-3.0-DE","CC-BY-NC-4.0","CC-BY-NC-ND-1.0","CC-BY-NC-ND-2.0","CC-BY-NC-ND-2.5","CC-BY-NC-ND-3.0","CC-BY-NC-ND-3.0-DE","CC-BY-NC-ND-3.0-IGO","CC-BY-NC-ND-4.0","CC-BY-NC-SA-1.0","CC-BY-NC-SA-2.0","CC-BY-NC-SA-2.0-DE","CC-BY-NC-SA-2.0-FR","CC-BY-NC-SA-2.0-UK","CC-BY-NC-SA-2.5","CC-BY-NC-SA-3.0","CC-BY-NC-SA-3.0-DE","CC-BY-NC-SA-3.0-IGO","CC-BY-NC-SA-4.0","CC-BY-ND-1.0","CC-BY-ND-2.0","CC-BY-ND-2.5","CC-BY-ND-3.0","CC-BY-ND-3.0-DE","CC-BY-ND-4.0","CC-BY-SA-1.0","CC-BY-SA-2.0","CC-BY-SA-2.0-UK","CC-BY-SA-2.1-JP","CC-BY-SA-2.5","CC-BY-SA-3.0","CC-BY-SA-3.0-AT","CC-BY-SA-3.0-DE","CC-BY-SA-3.0-IGO","CC-BY-SA-4.0","CC-PDDC","CC0-1.0","CDDL-1.0","CDDL-1.1","CDL-1.0","CDLA-Permissive-1.0","CDLA-Permissive-2.0","CDLA-Sharing-1.0","CECILL-1.0","CECILL-1.1","CECILL-2.0","CECILL-2.1","CECILL-B","CECILL-C","CERN-OHL-1.1","CERN-OHL-1.2","CERN-OHL-P-2.0","CERN-OHL-S-2.0","CERN-OHL-W-2.0","CFITSIO","check-cvs","checkmk","ClArtistic","Clips","CMU-Mach","CMU-Mach-nodoc","CNRI-Jython","CNRI-Python","CNRI-Python-GPL-Compatible","COIL-1.0","Community-Spec-1.0","Condor-1.1","copyleft-next-0.3.0","copyleft-next-0.3.1","Cornell-Lossless-JPEG","CPAL-1.0","CPL-1.0","CPOL-1.02","Cronyx","Crossword","CrystalStacker","CUA-OPL-1.0","Cube","curl","D-FSL-1.0","DEC-3-Clause","diffmark","DL-DE-BY-2.0","DL-DE-ZERO-2.0","DOC","Dotseqn","DRL-1.0","DRL-1.1","DSDP","dtoa","dvipdfm","ECL-1.0","ECL-2.0","EFL-1.0","EFL-2.0","eGenix","Elastic-2.0","Entessa","EPICS","EPL-1.0","EPL-2.0","ErlPL-1.1","etalab-2.0","EUDatagrid","EUPL-1.0","EUPL-1.1","EUPL-1.2","Eurosym","Fair","FBM","FDK-AAC","Ferguson-Twofish","Frameworx-1.0","FreeBSD-DOC","FreeImage","FSFAP","FSFAP-no-warranty-disclaimer","FSFUL","FSFULLR","FSFULLRWD","FTL","Furuseth","fwlw","GCR-docs","GD","GFDL-1.1-invariants-only","GFDL-1.1-invariants-or-later","GFDL-1.1-no-invariants-only","GFDL-1.1-no-invariants-or-later","GFDL-1.1-only","GFDL-1.1-or-later","GFDL-1.2-invariants-only","GFDL-1.2-invariants-or-later","GFDL-1.2-no-invariants-only","GFDL-1.2-no-invariants-or-later","GFDL-1.2-only","GFDL-1.2-or-later","GFDL-1.3-invariants-only","GFDL-1.3-invariants-or-later","GFDL-1.3-no-invariants-only","GFDL-1.3-no-invariants-or-later","GFDL-1.3-only","GFDL-1.3-or-later","Giftware","GL2PS","Glide","Glulxe","GLWTPL","gnuplot","GPL-1.0-only","GPL-1.0-or-later","GPL-2.0-only","GPL-2.0-or-later","GPL-3.0-only","GPL-3.0-or-later","Graphics-Gems","gSOAP-1.3b","gtkbook","HaskellReport","hdparm","Hippocratic-2.1","HP-1986","HP-1989","HPND","HPND-DEC","HPND-doc","HPND-doc-sell","HPND-export-US","HPND-export-US-modify","HPND-Fenneberg-Livingston","HPND-INRIA-IMAG","HPND-Kevlin-Henney","HPND-Markus-Kuhn","HPND-MIT-disclaimer","HPND-Pbmplus","HPND-sell-MIT-disclaimer-xserver","HPND-sell-regexpr","HPND-sell-variant","HPND-sell-variant-MIT-disclaimer","HPND-UC","HTMLTIDY","IBM-pibs","ICU","IEC-Code-Components-EULA","IJG","IJG-short","ImageMagick","iMatix","Imlib2","Info-ZIP","Inner-Net-2.0","Intel","Intel-ACPI","Interbase-1.0","IPA","IPL-1.0","ISC","ISC-Veillard","Jam","JasPer-2.0","JPL-image","JPNIC","JSON","Kastrup","Kazlib","Knuth-CTAN","LAL-1.2","LAL-1.3","Latex2e","Latex2e-translated-notice","Leptonica","LGPL-2.0-only","LGPL-2.0-or-later","LGPL-2.1-only","LGPL-2.1-or-later","LGPL-3.0-only","LGPL-3.0-or-later","LGPLLR","Libpng","libpng-2.0","libselinux-1.0","libtiff","libutil-David-Nugent","LiLiQ-P-1.1","LiLiQ-R-1.1","LiLiQ-Rplus-1.1","Linux-man-pages-1-para","Linux-man-pages-copyleft","Linux-man-pages-copyleft-2-para","Linux-man-pages-copyleft-var","Linux-OpenIB","LOOP","LPD-document","LPL-1.0","LPL-1.02","LPPL-1.0","LPPL-1.1","LPPL-1.2","LPPL-1.3a","LPPL-1.3c","lsof","Lucida-Bitmap-Fonts","LZMA-SDK-9.11-to-9.20","LZMA-SDK-9.22","Mackerras-3-Clause","Mackerras-3-Clause-acknowledgment","magaz","mailprio","MakeIndex","Martin-Birgmeier","McPhee-slideshow","metamail","Minpack","MirOS","MIT","MIT-0","MIT-advertising","MIT-CMU","MIT-enna","MIT-feh","MIT-Festival","MIT-Modern-Variant","MIT-open-group","MIT-testregex","MIT-Wu","MITNFA","MMIXware","Motosoto","MPEG-SSG","mpi-permissive","mpich2","MPL-1.0","MPL-1.1","MPL-2.0","MPL-2.0-no-copyleft-exception","mplus","MS-LPL","MS-PL","MS-RL","MTLL","MulanPSL-1.0","MulanPSL-2.0","Multics","Mup","NAIST-2003","NASA-1.3","Naumen","NBPL-1.0","NCGL-UK-2.0","NCSA","Net-SNMP","NetCDF","Newsletr","NGPL","NICTA-1.0","NIST-PD","NIST-PD-fallback","NIST-Software","NLOD-1.0","NLOD-2.0","NLPL","Nokia","NOSL","Noweb","NPL-1.0","NPL-1.1","NPOSL-3.0","NRL","NTP","NTP-0","O-UDA-1.0","OCCT-PL","OCLC-2.0","ODbL-1.0","ODC-By-1.0","OFFIS","OFL-1.0","OFL-1.0-no-RFN","OFL-1.0-RFN","OFL-1.1","OFL-1.1-no-RFN","OFL-1.1-RFN","OGC-1.0","OGDL-Taiwan-1.0","OGL-Canada-2.0","OGL-UK-1.0","OGL-UK-2.0","OGL-UK-3.0","OGTSL","OLDAP-1.1","OLDAP-1.2","OLDAP-1.3","OLDAP-1.4","OLDAP-2.0","OLDAP-2.0.1","OLDAP-2.1","OLDAP-2.2","OLDAP-2.2.1","OLDAP-2.2.2","OLDAP-2.3","OLDAP-2.4","OLDAP-2.5","OLDAP-2.6","OLDAP-2.7","OLDAP-2.8","OLFL-1.3","OML","OpenPBS-2.3","OpenSSL","OpenSSL-standalone","OpenVision","OPL-1.0","OPL-UK-3.0","OPUBL-1.0","OSET-PL-2.1","OSL-1.0","OSL-1.1","OSL-2.0","OSL-2.1","OSL-3.0","PADL","Parity-6.0.0","Parity-7.0.0","PDDL-1.0","PHP-3.0","PHP-3.01","Pixar","Plexus","pnmstitch","PolyForm-Noncommercial-1.0.0","PolyForm-Small-Business-1.0.0","PostgreSQL","PSF-2.0","psfrag","psutils","Python-2.0","Python-2.0.1","python-ldap","Qhull","QPL-1.0","QPL-1.0-INRIA-2004","radvd","Rdisc","RHeCos-1.1","RPL-1.1","RPL-1.5","RPSL-1.0","RSA-MD","RSCPL","Ruby","SAX-PD","SAX-PD-2.0","Saxpath","SCEA","SchemeReport","Sendmail","Sendmail-8.23","SGI-B-1.0","SGI-B-1.1","SGI-B-2.0","SGI-OpenGL","SGP4","SHL-0.5","SHL-0.51","SimPL-2.0","SISSL","SISSL-1.2","SL","Sleepycat","SMLNJ","SMPPL","SNIA","snprintf","softSurfer","Soundex","Spencer-86","Spencer-94","Spencer-99","SPL-1.0","ssh-keyscan","SSH-OpenSSH","SSH-short","SSLeay-standalone","SSPL-1.0","SugarCRM-1.1.3","Sun-PPP","SunPro","SWL","swrule","Symlinks","TAPR-OHL-1.0","TCL","TCP-wrappers","TermReadKey","TGPPL-1.0","TMate","TORQUE-1.1","TOSL","TPDL","TPL-1.0","TTWL","TTYP0","TU-Berlin-1.0","TU-Berlin-2.0","UCAR","UCL-1.0","ulem","UMich-Merit","Unicode-3.0","Unicode-DFS-2015","Unicode-DFS-2016","Unicode-TOU","UnixCrypt","Unlicense","UPL-1.0","URT-RLE","Vim","VOSTROM","VSL-1.0","W3C","W3C-19980720","W3C-20150513","w3m","Watcom-1.0","Widget-Workshop","Wsuipa","WTFPL","X11","X11-distribute-modifications-variant","Xdebug-1.03","Xerox","Xfig","XFree86-1.1","xinetd","xkeyboard-config-Zinoviev","xlock","Xnet","xpp","XSkat","YPL-1.0","YPL-1.1","Zed","Zeeff","Zend-2.0","Zimbra-1.3","Zimbra-1.4","Zlib","zlib-acknowledgement","ZPL-1.1","ZPL-2.0","ZPL-2.1"]

Trained_model_references = ["Falk et al. Nature Methods 2019", "Ronneberger et al. arXiv in 2015", "Lucas von Chamier et al. biorXiv 2020"]
Trained_model_DOI = ["https://doi.org/10.1038/s41592-018-0261-2","https://doi.org/10.1007/978-3-319-24574-4_28", "https://doi.org/10.1101/2020.03.20.000133"]

# Training data
# ---------------------------------------
#@markdown ##Include information about training data (optional):
include_training_data = True #@param {type: "boolean"}
#@markdown ### - If it is published in the BioImage Model Zoo, please, provide the ID
data_from_bioimage_model_zoo = False #@param {type: "boolean"}
training_data_ID = ''#@param {type:"string"}
#@markdown ### - If not, please provide the URL tot he data and a short description
training_data_source = 'https://www.epfl.ch/labs/cvlab/data/data-em/'#@param {type:"string"}
training_data_description = 'As said in the webpage, the dataset represents a section taken from the CA1 hippocampus region of the brain. The resolution of each voxel is approximately 5x5x5nm.'#@param {type:"string"}

# Add input image information
# ---------------------------------------
#@markdown ##Choose if you want to threshold the network output and if you want to use the best threshold calculated before.
apply_threshold = True #@param {type: "boolean"}
Use_The_Best_Average_Threshold = True #@param {type:"boolean"}
#@markdown ###If not, please input:
threshold = 210 #@param {type:"number"}
if Use_The_Best_Average_Threshold:
    threshold = best_thresh
## project it into the (0,1) range to operate with it in the bioimageio.core
threshold = float(threshold) / 255

#@markdown ##Introduce the voxel size (pixel size for each Z-slice and the distance between Z-slices) (in microns) of the image provided as an example of the model processing:
# information about the example image
PixelSize = 1 #@param {type:"number"}
Zdistance = 1 #@param {type:"number"}

#@markdown ##Do you want to choose the example test image?
default_example_image = True #@param {type:"boolean"}

#@markdown ###If not, please input:
Test_input_path =  "" #@param {type:"string"}

if default_example_image:
    Test_input_path = testing_source
# Check that the base_path is on the path and otherwise add it
Test_input_path = check_base_path(base_path, Test_input_path)

# Load model parameters
# ---------------------------------------
def last_chars(x):
    return(x[-11:])
try:
    ckpt_dir_list = glob(full_QC_model_path + '/ckpt/*')
    ckpt_dir_list.sort(key=last_chars)
    last_ckpt_path = ckpt_dir_list[0]
    print('Predicting from checkpoint:', os.path.basename(last_ckpt_path))
except IndexError:
    raise CheckpointError('No previous checkpoints were found, please retrain model.')
params = pd.read_csv(os.path.join(full_QC_model_path, 'params.csv'), names=['val'], header=0, index_col=0)

# Load the model and process the example image
# ---------------------------------------
model = Unet3D(shape=params.loc['training_shape', 'val'])
model.model.load_weights(last_ckpt_path)

# ------------- Execute bioimage model zoo configuration ------------
# Create a model without compilation so it can be used in any other environment.
# Remove the custom loss function from the model, so that it can be used outside of this notebook
unet = Model(model.model.input, model.model.output)
unet_weights_path = os.path.join(full_QC_model_path, 'keras_weights.hdf5')
unet.save(unet_weights_path)

unet_tf_weights = bioimageio_spec.KerasHdf5WeightsDescr(source=unet_weights_path,
                                                        tensorflow_version=tf.__version__)
unet_weights = bioimageio_spec.WeightsDescr(keras_hdf5=unet_tf_weights)

# Batch should never be constrained
shape = [sh for sh in unet.input.shape]
assert shape[0] is None

# Batch is set to 1 for bioimage.io
shape[0] = 1

assert all(sh is not None for sh in shape)  # make sure all other shapes are fixed
# Prepare the author/maintainer/packager
# ---------------------------------------

author_name_list = [e.strip() for e in Trained_model_authors.split(',')]
author_affiliation_list = [e.strip() for e in Trained_model_authors_affiliation.split(',')]

authors = []
for author, affiliation in zip(author_name_list, author_affiliation_list):
  authors.append(bioimageio_spec.Author(name=author, affiliation=affiliation))

assert len(author_name_list) >= Trained_model_packager, "Author list has less authors than the packager number inserted."
packager = [authors[Trained_model_packager-1]]

if Trained_model_maintainer_name != "Maintainer Name":
  # The user specified a new person as a maintainer
  maintainer = [bioimageio_spec.Maintainer(github_user=Trained_model_maintainer_github,
                                           name=Trained_model_maintainer_name)]
else:
  # The user chose an author as a maintainer
  assert len(author_name_list) >= Trained_model_maintainer_number, "Author list has less authors than the maintainer number inserted, no maintainer name will be added."
  maintainer = [bioimageio_spec.Maintainer(github_user=Trained_model_maintainer_github,
                                           name=author_name_list[Trained_model_maintainer_number-1],
                                           affiliation=author_affiliation_list[Trained_model_maintainer_number-1])]

# License
# ---------------------------------------

# I would recommend using CCBY-4 as licencese
license = Trained_model_license

# Output folder to export the model
# ---------------------------------------

# where to save the model
output_root = os.path.join(full_QC_model_path, Trained_model_name + '.bioimage.io.model')
os.makedirs(output_root, exist_ok=True)
output_path = os.path.join(output_root, f"{Trained_model_name}.zip")

# Create README.md file
# ---------------------------------------

# create a markdown readme with information
documentation_path = os.path.join(output_root, "README.md")
with open(documentation_path, "w") as f:
  f.write("Visit https://github.com/HenriquesLab/ZeroCostDL4Mic/wiki \n\n This was an automatically generated README.md. \n\n")

# Citations
# ---------------------------------------

assert len(Trained_model_DOI) == len(Trained_model_references)
citations = [{'text': text, 'doi': doi.replace('https://doi.org/', '')} for text, doi in zip(Trained_model_references, Trained_model_DOI)]
citation_spec = [bioimageio_spec.CiteEntry(**c) for c in citations]

# Training data
# ---------------------------------------

if include_training_data:
    if data_from_bioimage_model_zoo:
      training_data = {"id": training_data_ID}
    else:
      with open(documentation_path, "a") as f:
        f.write(f'Training data: {training_data_source} \n\n and description: {training_data_description} \n\n')
        training_data = None
else:
    training_data = None

# Preprocessing
# ---------------------------------------

min_percentile = 0
max_percentile = 100

bmz_preprocess = [bioimageio_spec.EnsureDtypeDescr(kwargs=bioimageio_spec.EnsureDtypeKwargs(dtype="float32"))]
bmz_preprocess.append(bioimageio_spec.ScaleRangeDescr(kwargs=bioimageio_spec.ScaleRangeKwargs(axes = ['x','y', 'z', 'channel'],
                                                                                              min_percentile = min_percentile ,
                                                                                              max_percentile = max_percentile)))

# Test input image
# ---------------------------------------

# load the input image, crop it if necessary, and save as numpy file
# The crop will be centered to get an image with some content.
test_img = tifffile.imread(Test_input_path).T

x_size = int(test_img.shape[0]/2)
x_size = x_size-int(shape[1]/2)

y_size = int(test_img.shape[1]/2)
y_size = y_size-int(shape[2]/2)

z_size = int(test_img.shape[2]/2)
z_size = z_size-int(shape[3]/2)

assert test_img.ndim == 3
test_img = test_img[x_size : x_size + shape[1],
                    y_size : y_size + shape[2],
                    z_size : z_size + shape[3]]

assert list(test_img.shape) == shape[1:4], f"{test_img.shape}, {shape[1:4]}"
# Save the test image
test_input_numpy_path = os.path.join(output_root, "test_input.npy")
test_output_numpy_path = os.path.join(base_path, "temp_test_output.npy")

np.save(test_input_numpy_path, test_img[None, ..., None].astype(np.float32))  # add batch and channel axis
np.save(test_output_numpy_path, test_img[None, ..., None].astype(np.float32))  # add batch and channel axis

'''
# Normalize the image before adding batch and channel dimensions
eps = 1e-6 # Same epsilon value as in the bioimageio.core library
n = test_img - np.min(test_img)
d = np.max(test_img) - np.min(test_img) + eps
test_img = n/d
test_img = test_img[None, ..., None]
test_prediction = unet.predict(test_img)

# run prediction on the input image and save the result as expected output
if apply_threshold:
  # Normalise the values of the test image

  n = test_prediction - np.min(test_prediction)
  d = np.max(test_prediction) - np.min(test_prediction) + 1e-6
  test_prediction = n/d

  test_prediction = (np.squeeze(test_prediction) > threshold).astype(np.uint8)
else:
  test_prediction = np.squeeze(test_prediction)
assert test_prediction.ndim == 3
test_prediction = test_prediction[None, ..., None]  # add batch and channel axis
test_out_path = os.path.join(output_root, "test_output.npy")
np.save(test_out_path, test_prediction)
'''

# Input specs
# ---------------------------------------

# Create the channel names
channel_names = [f'channel{idx}' for idx in range(shape[4])]

# Create the input tensor
input_tensor_axes = [bioimageio_spec.BatchAxis(id='batch',
                                               description='',
                                               type='batch',
                                               size=shape[0]),
                     bioimageio_spec.SpaceInputAxis(id='x',
                                                    description='',
                                                    type='space',
                                                    size=shape[1],
                                                    unit='micrometer',
                                                    scale=PixelSize,
                                                    concatenable=False),
                     bioimageio_spec.SpaceInputAxis(id='y',
                                                    description='',
                                                    type='space',
                                                    size=shape[2],
                                                    unit='micrometer',
                                                    scale=PixelSize,
                                                    concatenable=False),
                     bioimageio_spec.SpaceInputAxis(id='z',
                                                    description='',
                                                    type='space',
                                                    size=shape[3],
                                                    unit='micrometer',
                                                    scale=Zdistance,
                                                    concatenable=False),
                     bioimageio_spec.ChannelAxis(id='channel',
                                                 description='',
                                                 type='channel',
                                                 channel_names=channel_names),
                    ]

input_tensor = bioimageio_spec.InputTensorDescr(id=bioimageio_spec.TensorId('input0'),
                                                description= 'This is the test input tensor created from the example image.',
                                                axes=input_tensor_axes,
                                                test_tensor = bioimageio_spec.FileDescr(source = test_input_numpy_path),
                                                preprocessing = bmz_preprocess
                                                )

# Postprocessing
# ---------------------------------------

bmz_postprocess = []

if apply_threshold:
  print("The model output will be thresholded.")
  bmz_postprocess.append(bioimageio_spec.ScaleRangeDescr(kwargs=bioimageio_spec.ScaleRangeKwargs(axes = ['x','y', 'z', 'channel'],
                                                                                              min_percentile = 0 ,
                                                                                              max_percentile = 100)))
  bmz_postprocess.append(bioimageio_spec.BinarizeDescr(kwargs=bioimageio_spec. BinarizeKwargs(threshold = threshold)))
else:
  print("The model output will not be thresholded.")

bmz_postprocess.append(bioimageio_spec.EnsureDtypeDescr(kwargs=bioimageio_spec.EnsureDtypeKwargs(dtype="float32")))


# Output specs
# ---------------------------------------

# Create the output tensor
output_tensor_axes = [bioimageio_spec.BatchAxis(id='batch',
                                                description='',
                                                type='batch',
                                                size=None),
                      bioimageio_spec.SpaceOutputAxis(id='x',
                                                      description='',
                                                      type='space',
                                                      unit='micrometer',
                                                      scale=1.0, # consider changing it if the input has more than one channel
                                                      size=bioimageio_spec.SizeReference(tensor_id=bioimageio_spec.TensorId('input0'),
                                                                                          axis_id='x',
                                                                                          offset=0)),
                      bioimageio_spec.SpaceOutputAxis(id='y',
                                                      description='',
                                                      type='space',
                                                      unit='micrometer',
                                                      scale=1.0, # consider changing it if the input has more than one channel
                                                      size=bioimageio_spec.SizeReference(tensor_id=bioimageio_spec.TensorId('input0'),
                                                                                          axis_id='y',
                                                                                          offset=0)),
                      bioimageio_spec.SpaceOutputAxis(id='z',
                                                      description='',
                                                      type='space',
                                                      unit='micrometer',
                                                      scale=1.0, # consider changing it if the input has more than one channel
                                                      size=bioimageio_spec.SizeReference(tensor_id=bioimageio_spec.TensorId('input0'),
                                                                                          axis_id='z',
                                                                                          offset=0)),
                      bioimageio_spec.ChannelAxis(id='channel',
                                                  description='',
                                                  type='channel',
                                                  channel_names=channel_names),
                      ]

output_tensor = bioimageio_spec.OutputTensorDescr(id=bioimageio_spec.TensorId("output0"),
                                                  axes=output_tensor_axes,
                                                  test_tensor = bioimageio_spec.FileDescr(source = test_output_numpy_path),
                                                  postprocessing = bmz_postprocess,
                                                  )

# Create the cover
# ---------------------------------------

cover_path = Test_input_path

# Attach the QC report to the model (if it exists)
# ---------------------------------------

attachments = []
qc_path = os.path.join(full_QC_model_path, 'Quality Control', 'training_evaluation.csv')
if os.path.exists(qc_path):
  attachments.append(FileDescr(source = qc_path))


# Build the bioimage model zoo model
# ---------------------------------------

for i in range(2):
  # We create the model, process the input image and create the model again with the correct output.
  model_description = bioimageio_spec.ModelDescr(name=Trained_model_name,
                                                description=Trained_model_description,
                                                covers=[cover_path],
                                                authors=authors,
                                                attachments=attachments,
                                                cite=citation_spec,
                                                license=license,
                                                maintainers=maintainer,
                                                packaged_by=packager,
                                                tags=['zerocostdl4mic', 'deepimagej',
                                                      'segmentation', 'unet', '3D'],
                                                documentation= documentation_path,
                                                inputs=[input_tensor],
                                                outputs=[output_tensor],
                                                weights=unet_weights,
                                                training_data=training_data,
                                              )

  if i == 0:
    # Define the new input sample (taken from the new model description)
    new_input_paths = {ipt.id: download(ipt.test_tensor).path for ipt in model_description.inputs}

    # The prediction pipeline expects a Sample object from bioimageio.core
    input_sample = create_sample_for_model(
        model=model_description, inputs=new_input_paths, sample_id="my_demo_sample"
    )

    # Create the new prediction
    prediction = bioimageio_core.predict(model=model_description, inputs=input_sample)

    # Save the new prediction on a NumPy file
    new_output_path = os.path.join(output_root, "test_output.npy")

    # Delete the file if exists to avoid SHA256 mismatch
    if os.path.exists(test_output_numpy_path):
      os.remove(test_output_numpy_path)

    prediction_tensor = prediction.members["output0"].data
    np.save(new_output_path, prediction_tensor)

    # Create the output tensor description

    output_tensor = bioimageio_spec.OutputTensorDescr(id=bioimageio_spec.TensorId("output0"),
                                                      axes=output_tensor_axes,
                                                      test_tensor = bioimageio_spec.FileDescr(source = new_output_path),
                                                      postprocessing = bmz_postprocess,
                                                     )

    # Clean first model
    del model_description

# Test the model
# ---------------------------------------

# Using the validation context, it will reset the know files
# Avoiding the SHA256 mistmach errors
with bioimageio.spec.ValidationContext():
  summary = bioimageio_core.test_model(model_description)
  summary.display()

  success = summary.status == "passed"

success = True

if success:
  with bioimageio.spec.ValidationContext():
    # In case it has passed the test, save the bioimage.io model with the correct format
    save_bioimageio_package(model_description, output_path=Path(output_path))
    print("\nThe bioimage.io model was successfully exported to", output_path)
else:
  print("\nThe bioimage.io model was exported to", output_path)
  print("Some tests of the model did not work! You can still download and test the model.")
  print("You can still download and test the model, but it may not work as expected.")




'''
# export the model with keras weihgts
build_model(
    weight_uri=weight_path,
    test_inputs=[test_in_path],
    test_outputs=[test_out_path],
    name=Trained_model_name,
    description=Trained_model_description,
    authors=authors,
    tags=['zerocostdl4mic', 'deepimagej', 'segmentation', 'tem', 'unet'],
    license=Trained_model_license,
    documentation=readme_path,
    cite=citations,
    output_path=output_path,
    add_deepimagej_config=True,
    tensorflow_version=tf.__version__,
    attachments=attachments,
    training_data = training_data,
    **kwargs
)

# convert the keras weights to tensorflow and add them to the model
tf_weight_path = os.path.join(full_qc_model_path, "tf_weights")
# we need to make sure that the tf weight folder does not exist
if os.path.exists(tf_weight_path):
  rmtree(tf_weight_path)
convert_weights_to_tensorflow_saved_model_bundle(output_path, tf_weight_path + ".zip")
add_weights(output_path, tf_weight_path + ".zip", output_path, tensorflow_version=tf.__version__)

# check that the model works for keras and tensorflow
res = test_model(output_path, weight_format="keras_hdf5")
success = True
if res[-1]["error"] is not None:
  success = False
  print("test-model failed for keras weights:", res[-1]["error"])

res = test_model(output_path, weight_format="tensorflow_saved_model_bundle")
if res[-1]["error"] is not None:
  success = False
  print("test-model failed for tensorflow weights:", res[-1]["error"])

if success:
  print("The bioimage.io model was successfully exported to", output_path)
else:
  print("The bioimage.io model was exported to", output_path)
  print("Some tests of the model did not work! You can still download and test the model.")
  print("You can still download and test the model, but it may not work as expected.")
''';

# **6. Using the trained model**

---

<font size = 4>Once sufficient performance of the trained model has been established using Section 5, the network can be used to segment unseen volumetric data.

## **6.1. Generate predictions from unseen dataset**
---

<font size = 4>The most recently trained model can now be used to predict segmentation masks on unseen images. If you want to use an older model, leave `model_path`  blank. Predicted output images are saved in `output_path` as Image-J compatible TIFF files.

## **Prediction parameters**

* <font size = 4>**`source_path`** specifies the location of the source
image volume.

* <font size = 4>**`output_directory`** specified the directory where the output predictions are stored.

* <font size = 4>**`binary_target`** should be chosen if the network is trained to predict binary segmentation masks.

* <font size = 4>**`threshold`** can be calculated in Section 5 and is used to generate binary masks from the predictions.

* <font size = 4>**`big_tiff`** should be chosen if the expected prediction exceeds 4GB. The predictions will be saved using the BigTIFF format. Beware that this might substantially reduce the prediction speed. *Default: False*

* <font size = 4>**`prediction_depth`** is only relevant if the prediction is saved as a BigTIFF. The prediction will not be performed in one go to not deplete the memory resources. Instead, the prediction is iteratively performed on a subset of the entire volume  with shape `(source.shape[0], source.shape[1], prediction_depth)`. *Default: 32*

* <font size = 4>**`model_path`** specifies the path to a model other than the most  recently trained.

In [ ]:
#@markdown ## Download example volume

#@markdown <font size = 4> This can take up to an hour.
def download_from_url(url, save_as):
    file_url = url
    r = requests.get(file_url, stream=True)

    with open(save_as, 'wb') as file:
        for block in tqdm(r.iter_content(chunk_size = 1024), desc = 'Downloading ' + os.path.basename(save_as), total=3275073, ncols=1000):
            if block:
                file.write(block)

# Download example volume
os.makedirs('example_dataset', exist_ok=True)
download_from_url('https://documents.epfl.ch/groups/c/cv/cvlab-unit/www/data/%20ElectronMicroscopy_Hippocampus/volumedata.tif', 'example_dataset/volumedata.tif')

In [ ]:
#@markdown ### Provide the path to your dataset and to the folder where the predictions are saved, then run the cell to predict outputs from your unseen images.

source_path = "" #@param {type:"string"}
output_directory = "" #@param {type:"string"}

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

output_path = os.path.join(output_directory, os.path.splitext(os.path.basename(source_path))[0] + '_predicted.tif')
#@markdown ###Prediction parameters:

binary_target = True #@param {type:"boolean"}

save_probability_map = False #@param {type:"boolean"}

#@markdown <font size = 3>Determine best threshold in Section 5.2.

use_calculated_threshold = True #@param {type:"boolean"}
threshold =  200#@param {type:"number"}

# Tifffile library issues means that images cannot be appended to
#@markdown <font size = 3>Choose if prediction file exceeds 4GB or if input file is very large (above 2GB). Image volume saved as BigTIFF.
big_tiff = False #@param {type:"boolean"}

#@markdown <font size = 3>Reduce `prediction_depth` if runtime runs out of memory during prediction. Only relevant if prediction saved as BigTIFF

prediction_depth =  32#@param {type:"number"}

#@markdown ###Model to be evaluated
#@markdown <font size = 3>If left blank, the latest model defined in Section 5 will be evaluated

full_model_path_ = "" #@param {type:"string"}

if len(full_model_path_) == 0:
    full_model_path_ = full_QC_model_path



# Load parameters
params = pd.read_csv(os.path.join(full_model_path_, 'params.csv'), names=['val'], header=0, index_col=0)
model = Unet3D(shape=params.loc['training_shape', 'val'])

if use_calculated_threshold:
    threshold = best_thresh

def last_chars(x):
    return(x[-11:])

try:
    ckpt_dir_list = glob(full_model_path_ + '/ckpt/*')
    ckpt_dir_list.sort(key=last_chars)
    last_ckpt_path = ckpt_dir_list[0]
    print('Predicting from checkpoint:', os.path.basename(last_ckpt_path))
except IndexError:
    raise CheckpointError('No previous checkpoints were found, please retrain model.')

src = tifffile.imread(source_path)

if src.nbytes >= 4e9:
    big_tiff = True
    print('The source file exceeds 4GB in memory, prediction will be saved as BigTIFF!')

if binary_target:
    if not big_tiff:
        prediction = model.predict(src, last_ckpt_path, downscaling=params.loc['downscaling', 'val'], true_patch_size=params.loc['true_patch_size', 'val'])
        prediction = np.interp(prediction, (prediction.min(), prediction.max()), (0, 255))
        prediction = (prediction > threshold).astype('float32')
        prediction = prediction.reshape(prediction.shape[0], 1, prediction.shape[1], prediction.shape[2])

        tifffile.imwrite(output_path, prediction, imagej=True)

    else:
        with tifffile.TiffWriter(output_path, bigtiff=True) as tif:
            for i in tqdm(range(0, src.shape[0], prediction_depth)):
                prediction = model.predict(src, last_ckpt_path, z_range=(i,i+prediction_depth), downscaling=params.loc['downscaling', 'val'], true_patch_size=params.loc['true_patch_size', 'val'])
                prediction = np.interp(prediction, (prediction.min(), prediction.max()), (0, 255))
                prediction = (prediction > threshold).astype('float32')

                for j in range(prediction.shape[0]):
                    tif.write(prediction[j])

if not binary_target or save_probability_map:
    if not binary_target:
        prob_map_path = output_path
    else:
        prob_map_path = os.path.splitext(output_path)[0] + '_prob_map.tif'

    if not big_tiff:
        prediction = model.predict(src, last_ckpt_path, downscaling=params.loc['downscaling', 'val'], true_patch_size=params.loc['true_patch_size', 'val'])
        prediction = np.interp(prediction, (prediction.min(), prediction.max()), (0, 255))
        prediction = prediction.reshape(prediction.shape[0], 1, prediction.shape[1], prediction.shape[2])
        tifffile.imwrite(prob_map_path, prediction.astype('float32'), imagej=True)

    else:
        with tifffile.TiffWriter(prob_map_path, bigtiff=True) as tif:
            for i in tqdm(range(0, src.shape[0], prediction_depth)):
                prediction = model.predict(src, last_ckpt_path, z_range=(i,i+prediction_depth), downscaling=params.loc['downscaling', 'val'], true_patch_size=params.loc['true_patch_size', 'val'])
                prediction = np.interp(prediction, (prediction.min(), prediction.max()), (0, 255))

                for j in range(prediction.shape[0]):
                    tif.write(prediction[j])

print('Predictions saved as', output_path)

src_volume = tifffile.imread(source_path)
pred_volume = tifffile.imread(output_path)

def scroll_in_z(z):

    f=plt.figure(figsize=(25,5))
    plt.subplot(1,2,1)
    plt.imshow(src_volume[z-1], cmap='gray')
    plt.title('Source (z = ' + str(z) + ')', fontsize=15)
    plt.axis('off')

    plt.subplot(1,2,2)
    plt.imshow(pred_volume[z-1], cmap='magma')
    plt.title('Prediction (z = ' + str(z) + ')', fontsize=15)
    plt.axis('off')

interact(scroll_in_z, z=widgets.IntSlider(min=1, max=src_volume.shape[0], step=1, value=0));


## **6.2. Download your predictions**
---

<font size = 4>**Store your data** and ALL its results elsewhere by downloading it from Google Drive and after that clean the original folder tree (datasets, results, trained model etc.) if you plan to train or use new networks. Please note that the notebook will otherwise **OVERWRITE** all files which have the same name.

# **7. Version log**

---

<font size = 4>**v2.3.1**:  

*   Updated Bioimage.IO model export to latest version (core-0.6.9, spec-0.5.3.2)
*   Fixed model importation from Bioimage.IO
*   Fixed Tensorflow version to 2.15
*   Bug fixes

<font size = 4>**v2.1.3**:  

*   Changed default voxel size for Bioimage Model Zoo format export

<font size = 4>**v2.1.2**:  

*   Prediction results should now correctly be saved as volumes
*   Updated deprecated code for saving BigTIFF volumes

<font size = 4>**v2.1.1**:  

*   Replaced all absolute pathing with relative pathing


<font size = 4>**v2.1**:
* Updated notebook to use with TensorFlow 2.11
* The full requirements file is exported to replicate the environment
* Bug in the preprocessing to be compatible with the BioImage Model Zoo is solved.

* Installs `fpdf2`, curates the format of the pdf and fixes bugs in `keras` versioning writting.

---

# **Thank you for using 3D U-Net!**